In [14]:
# Imports
import pandas as pd
import numpy as np
import os

# CBS Imports
import cbsodata

In [15]:
# Set Viewing Options
pd.set_option('display.width', 5000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Helper functions

In [16]:
def convert_duplicate_columns(names):
    names = list(names)  # so we can index
    counts = {}

    for i, col in enumerate(names):
        cur_count = counts.get(col, 0)

        if cur_count > 0:
            names[i] = '%s.%d' % (col, cur_count)

        counts[col] = cur_count + 1

    return names

In [17]:
def clean_open_data(df: pd.DataFrame) -> pd.DataFrame:

    # Remove leading and trailing spaces from object (str) columns
    df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.str.strip())

    # Remove trailing underscore and number used by CBS. Regex finds the last occurrence of "_12" (any number) and removes it. The following replace removes all remaining underscores.
    df.columns = df.columns.str.replace("(\_[0-9]+)(?!.*\_[0-9]+)", "", regex=True).str.replace("_", "")

    # Check for duplicates and give each subsequent number trailing the column name .e.g. original = Appartement, first duplicate = Appartement.1 
    # df.columns = pd.io.parsers.base_parser.ParserBase({'names':df.columns, 'usecols':None})._maybe_dedup_names(df.columns)
    df.columns = convert_duplicate_columns(df.columns)

    # Limit DataFrame to "buurt" level data
    if "SoortRegio" in df.columns:
        df = df[df["SoortRegio"] == "Buurt"]

    # Reset Index
    df = df.reset_index(drop=True)

    # Format numbers as interger wherever possible, otherwise format as a float
    df = df.apply(lambda x: pd.to_numeric(x, errors="ignore"))

    return df

In [18]:
def rename_c_kerncijfers(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns={"Appartement": "EnergieVerbruikAppartement",
                            "Appartement.1": "AardgasVerbruikAppartement",
                            "Tussenwoning": "EnergieVerbruikTussenwoning",
                            "Tussenwoning.1": "AardgasVerbruikTussenwoning",
                            "Hoekwoning": "EnergieVerbruikHoekwoning",
                            "Hoekwoning.1": "AardgasVerbruikHoekwoning",
                            "TweeOnderEenKapWoning": "EnergieVerbruikTweeOnderEenKapWoning",
                            "TweeOnderEenKapWoning.1": "AardgasVerbruikTweeOnderEenKapWoning",
                            "VrijstaandeWoning": "EnergieVerbruikVrijstaandeWoning",
                            "VrijstaandeWoning.1": "AardgasVerbruikVrijstaandeWoning",
                            "Huurwoning": "EnergieVerbruikHuurwoning",
                            "Huurwoning.1": "AardgasVerbruikHuurwoning",
                            "EigenWoning": "EnergieVerbruikEigenWoning",
                            "EigenWoning.1": "AardgasVerbruikEigenWoning"})

In [19]:
# Read in KoppelTabel
koppel_df_2019_2020 = pd.read_excel('../../Data/Location/Auxilliary/Koppel_tabel.xlsx', sheet_name="2019-2020")
koppel_df_2020_2021 = pd.read_excel('../../Data/Location/Auxilliary/Koppel_tabel.xlsx', sheet_name="2020-2021")
koppel_df_2021_2022 = pd.read_excel('../../Data/Location/Auxilliary/Koppel_tabel.xlsx', sheet_name="2021-2022")


koppel_dict = {}

for koppel_df in [koppel_df_2019_2020, koppel_df_2020_2021, koppel_df_2021_2022]:
    
    old_year, new_year = koppel_df.columns[0], koppel_df.columns[4]
    
    # Remove the topheader which was empty except for 2 years, and put the first row as a header
    koppel_df.columns = koppel_df.iloc[0].values
    koppel_df = koppel_df[1:].reset_index(drop=True)

    # Create subdict per transition .e.g. 2019-2020, 2020-2021, which respectively consist of (key: year, value: pd.dataframe) pairs
    # Also remove wijken
    koppel_df.columns = convert_duplicate_columns(koppel_df.columns)
    koppel_dict[(old_year, new_year)] = koppel_df[~koppel_df["code buurt of wijk"].str.contains("WK")].reset_index(drop=True).copy()

In [20]:
def show_info(df_list: list[pd.DataFrame]) -> None:

    if len(df_list) == 1:
        df = df_list[0]
        nans = df.isna().sum().sum()
        print(f"Info Dataset {df.columns.name}")
        print("- Total Values:".ljust(40) + str(np.prod(df.shape)))
        print("- Total Missings:".ljust(40) + str(nans))
        print("- % Missing:".ljust(40) + str(round((nans / np.prod(df.shape)*100), 2)) + "%")
        print("- Dataset Dimensions:".ljust(40) + str(df.shape) + "\n")

    elif len(df_list) == 2:
        df_1, df_2 = df_list[0], df_list[1]
        nans_1, nans_2 = df_1.isna().sum().sum(), df_2.isna().sum().sum() 
        print(" Info Datasets".ljust(40) + df_1.columns.name.ljust(40) + df_2.columns.name + "\n")
        print("- Total Values:".ljust(40) + str(np.prod(df_1.shape)).ljust(40) + str(np.prod(df_2.shape)))
        print("- Total Missings:".ljust(40) + str(nans_1).ljust(40) + str(nans_2))
        print("- % Missing:".ljust(40) + str(round((nans_1 / np.prod(df_1.shape)*100), 2)).ljust(40) + str(round((nans_2 / np.prod(df_2.shape)*100), 2)))
        print("- Dataset Dimensions:".ljust(40) + str(df_1.shape).ljust(40) + str(df_2.shape) + "\n")

In [21]:
def convert_codering(codering: str, old_year: int, new_year: int) -> str:
    if codering in koppel_dict[tuple((old_year, new_year))]['code buurt of wijk'].values:
        new_value = koppel_dict[tuple((old_year, new_year))].loc[koppel_dict[tuple((old_year, new_year))]['code buurt of wijk'] == codering, "code buurt of wijk.1"].values[0]
        return new_value
    else:
        return codering

### Download Data

In [22]:
location_data_dict = {year: {} for year in range(2018, 2023)}

# 2018
location_data_dict[2018]["CBS Buurtkerncijfers"] = pd.DataFrame(cbsodata.get_data("84286NED"))

# 2019
location_data_dict[2019]["CBS Buurtkerncijfers"] = pd.DataFrame(cbsodata.get_data("84583NED"))
location_data_dict[2019]["CBS Regionaalkerncijfers"] = pd.DataFrame(cbsodata.get_data("70072NED", filters = "(Perioden eq '2019JJ00')"))

# 2020
location_data_dict[2020]["CBS Buurtkerncijfers"] = pd.DataFrame(cbsodata.get_data("84799NED"))
location_data_dict[2020]["CBS Regionaalkerncijfers"] = pd.DataFrame(cbsodata.get_data("70072NED", filters = "(Perioden eq '2020JJ00')"))
location_data_dict[2020]["CBS Voorzieningen"] = pd.DataFrame(cbsodata.get_data("84953NED"))
location_data_dict[2020]["CBS Gezondheid"] = pd.DataFrame(cbsodata.get_data("50090NED", catalog_url="dataderden.cbs.nl", filters = "(Perioden eq '2020JJ00') and (Leeftijd eq '20300')"))
location_data_dict[2020]["CBS Zonnepanelen"] = pd.DataFrame(cbsodata.get_data("85447NED"))
location_data_dict[2020]["Leefbarometer 3.0"] = pd.read_csv("https://data.overheid.nl/sites/default/files/dataset/f37c3649-cc52-4e48-a864-870ae42807a2/resources/Leefbaarometer%203.0%20-%20meting%202020%20-%20scores%20buurt.csv")

# 2021
location_data_dict[2021]["CBS Buurtkerncijfers"] = pd.DataFrame(cbsodata.get_data("85039NED"))
location_data_dict[2021]["CBS Regionaalkerncijfers"] = pd.DataFrame(cbsodata.get_data("70072NED", filters = "(Perioden eq '2021JJ00')"))
location_data_dict[2021]["CBS Voorzieningen"] = pd.DataFrame(cbsodata.get_data("85231NED"))

# 2022
location_data_dict[2022]["CBS Buurtkerncijfers"] = pd.DataFrame(cbsodata.get_data("85318NED"))
location_data_dict[2022]["CBS Regionaalkerncijfers"] = pd.DataFrame(cbsodata.get_data("70072NED", filters = "(Perioden eq '2022JJ00')"))
location_data_dict[2022]["CBS Overlast"] = pd.DataFrame(cbsodata.get_data("47024NED",  catalog_url="dataderden.cbs.nl", filters = "(Perioden eq '2022JJ00')"))
location_data_dict[2022]["CBS Misdrijven"] = pd.DataFrame(cbsodata.get_data("47018NED", catalog_url="dataderden.cbs.nl", filters = "(Perioden eq '2022JJ00')"))

### Store data locally

In [23]:
if not os.path.exists("../../Data/Location/CBS"):
    os.makedirs("../../Data/Location/CBS")
    
    for year in range(2018, 2023):
        os.makedirs(f"../../Data/Location/CBS/{year}")
        for name, df in location_data_dict[year].items():
            df.to_parquet(f"../../Data/Location/CBS/{year}/{name}.gzip", compression="gzip")

else:
    print("Folder Already Exists!")
    for year in range(2018, 2023):
        for name, df in location_data_dict[year].items():
            df.to_parquet(f"../../Data/Location/CBS/{year}/{name}.gzip", compression="gzip")

Folder Already Exists!


In [24]:
# Storage for yearly contribution to the data
contribution_dict = {}

---
# 2022

### Load DataFrames

- 2022 - CBS Kerncijfers  - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=85318NED
- 2022 - CBS Misdrijven   - https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47018NED
- 2022 - CBS Overlast     - https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47024NED
---

In [25]:
#====================================== Read & Clean the 2022 Datasets ============================================
kerncijfers_buurt_2022 =  rename_c_kerncijfers(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2022/CBS Buurtkerncijfers.gzip")))
misdrijven_2022 = clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2022/CBS Misdrijven.gzip"))
overlast_2022 = clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2022/CBS Overlast.gzip"))
#==================================================================================================================

#=========================================== Name 2022 Datasets ===================================================
kerncijfers_buurt_2022.columns.name = "Kerncijfers 2022"
misdrijven_2022.columns.name = "Misdrijven 2022"
overlast_2022.columns.name = "Overlast 2022"
#==================================================================================================================

---
### Merge
---
- 2022 - Overlast (Gefilterd op totale registraties overlast)
- 2022 - Kerncijfers

In [26]:
#========================= Filter Overlast 2022 Dataset and merge with kerncijfers ================================
overlast_2022 = overlast_2022[overlast_2022["Overlast"] == "Totaal registraties overlast"]
overlast_2022 = overlast_2022.groupby(["WijkenEnBuurten", "Gemeentenaam"], as_index=False).sum(numeric_only=True)
df_2022 = pd.merge(kerncijfers_buurt_2022, overlast_2022[["WijkenEnBuurten", "Gemeentenaam", "GeregistreerdeOverlast"]], on=["WijkenEnBuurten", "Gemeentenaam"], how="left").reset_index(drop=True)
df_2022.columns.name = "Dataset 2022"
#==================================================================================================================

In [27]:
#========================= Group all misdrijven and sum over geregistreerde aantallen =============================
misdrijven_grouped = (misdrijven_2022.groupby(["SoortMisdrijf", "WijkenEnBuurten", "Gemeentenaam", "Perioden"], as_index=False).agg({"GeregistreerdeMisdrijven":"sum"}))
misdrijven_grouped = misdrijven_grouped.drop_duplicates(["WijkenEnBuurten", "Gemeentenaam"])
#==================================================================================================================

---
### Merge
---
- 2022 - Misdrijven (Gecategoriseerd op type misdrijf en summed over geregistreerde misdrijven per buurt)
- 2022 - Kerncijfers + Overlast

In [28]:
#=============================== Categorize types of misdrijven en sum over them ==================================
misdrijf_dict = {"Inbraken":        ["1.1.1 Diefstal/inbraak woning", "1.1.2 Diefstal/inbraak box/garage/schuur", "1.2.2 Diefstal van motorvoertuigen",
                                     "1.2.3 Diefstal van brom-, snor-, fietsen", "1.2.4 Zakkenrollerij", "1.2.5 Diefstal af/uit/van ov. voertuigen",
                                     "2.5.1 Diefstal/inbraak bedrijven enz.", "2.5.2 Winkeldiefstal"],
                "Berovingen":       ["1.4.6 Straatroof", "1.4.7 Overval", "1.5.2 Diefstallen (water)"],
                "Geweld":           ["1.4.1 Zedenmisdrijf", "1.4.2 Moord, doodslag", "1.4.3 Openlijk geweld (persoon)", "1.4.4 Bedreiging", "1.4.5 Mishandeling"],
                "Vernielingen":     ["2.2.1 Vernieling cq. zaakbeschadiging"],
                "ordeverstoringen": ["2.1.1 Drugs/drankoverlast", "2.4.1 Burengerucht (relatieproblemen)", "2.4.2 Huisvredebreuk", "2.6.12 Ruimtelijke ordening",
                                     "2.6.9 Vuurwerk", "3.6.4 Aantasting openbare orde"]}

for key, value in misdrijf_dict.items():
    df = misdrijven_grouped[misdrijven_grouped["SoortMisdrijf"].isin(value)].groupby(["WijkenEnBuurten", "Gemeentenaam"], as_index=False).agg({"GeregistreerdeMisdrijven": "sum"}).rename(columns={"GeregistreerdeMisdrijven": f"{key}"})

    temp_df =  df.set_index(["WijkenEnBuurten", "Gemeentenaam"])
    temp_kerncijfers = df_2022.set_index(["WijkenEnBuurten", "Gemeentenaam"])

    df_2022 = pd.merge(temp_kerncijfers, temp_df, left_index=True, right_index=True, how="left").reset_index()

df_2022.columns.name = "Dataset 2022"
contribution_dict[2022] = np.prod(df_2022.shape) - df_2022.isna().sum().sum()
#==================================================================================================================


In [29]:
df_2022["Berovingen"] = df_2022["Berovingen"].fillna(0)
df_2022["Geweld"] = df_2022["Geweld"].fillna(0)
df_2022["Vernielingen"] = df_2022["Vernielingen"].fillna(0)
df_2022["ordeverstoringen"] = df_2022["ordeverstoringen"].fillna(0)
df_2022["Inbraken"] = df_2022["Inbraken"].fillna(0)
df_2022["GeregistreerdeOverlast"] = df_2022["GeregistreerdeOverlast"].fillna(0)

In [30]:
df_2022

Dataset 2022,WijkenEnBuurten,Gemeentenaam,ID,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AantalInwoners,Mannen,Vrouwen,k0Tot15Jaar,k15Tot25Jaar,k25Tot45Jaar,k45Tot65Jaar,k65JaarOfOuder,Ongehuwd,Gehuwd,Gescheiden,Verweduwd,WestersTotaal,NietWestersTotaal,Marokko,NederlandseAntillenEnAruba,Suriname,Turkije,OverigNietWesters,GeboorteTotaal,GeboorteRelatief,SterfteTotaal,SterfteRelatief,HuishoudensTotaal,Eenpersoonshuishoudens,HuishoudensZonderKinderen,HuishoudensMetKinderen,GemiddeldeHuishoudensgrootte,Bevolkingsdichtheid,Woningvoorraad,GemiddeldeWOZWaardeVanWoningen,PercentageEengezinswoning,PercentageMeergezinswoning,PercentageBewoond,PercentageOnbewoond,Koopwoningen,HuurwoningenTotaal,InBezitWoningcorporatie,InBezitOverigeVerhuurders,EigendomOnbekend,BouwjaarVoor2000,BouwjaarVanaf2000,GemiddeldElektriciteitsverbruikTotaal,EnergieVerbruikAppartement,EnergieVerbruikTussenwoning,EnergieVerbruikHoekwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,EnergieVerbruikHuurwoning,EnergieVerbruikEigenWoning,GemiddeldAardgasverbruikTotaal,AardgasVerbruikAppartement,AardgasVerbruikTussenwoning,AardgasVerbruikHoekwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AardgasVerbruikHuurwoning,AardgasVerbruikEigenWoning,PercentageWoningenMetStadsverwarming,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OpleidingsniveauHoog,Nettoarbeidsparticipatie,PercentageWerknemers,PercentageZelfstandigen,AantalInkomensontvangers,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,k40PersonenMetLaagsteInkomen,k20PersonenMetHoogsteInkomen,GemGestandaardiseerdInkomenVanHuish,k40HuishoudensMetLaagsteInkomen,k20HuishoudensMetHoogsteInkomen,HuishoudensMetEenLaagInkomen,HuishOnderOfRondSociaalMinimum,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,MediaanVermogenVanParticuliereHuish,PersonenPerSoortUitkeringBijstand,PersonenPerSoortUitkeringAO,PersonenPerSoortUitkeringWW,PersonenPerSoortUitkeringAOW,JongerenMetJeugdzorgInNatura,PercentageJongerenMetJeugdzorg,WmoClienten,WmoClientenRelatief,BedrijfsvestigingenTotaal,ALandbouwBosbouwEnVisserij,BFNijverheidEnEnergie,GIHandelEnHoreca,HJVervoerInformatieEnCommunicatie,KLFinancieleDienstenOnroerendGoed,MNZakelijkeDienstverlening,OQOverheidOnderwijsEnZorg,RUCultuurRecreatieOverigeDiensten,PersonenautoSTotaal,PersonenautoSBrandstofBenzine,PersonenautoSOverigeBrandstof,PersonenautoSPerHuishouden,PersonenautoSNaarOppervlakte,Motorfietsen,AfstandTotHuisartsenpraktijk,AfstandTotGroteSupermarkt,AfstandTotKinderdagverblijf,AfstandTotSchool,ScholenBinnen3Km,OppervlakteTotaal,OppervlakteLand,OppervlakteWater,MeestVoorkomendePostcode,Dekkingspercentage,MateVanStedelijkheid,Omgevingsadressendichtheid,GeregistreerdeOverlast,Inbraken,Berovingen,Geweld,Vernielingen,ordeverstoringen
0,'n Oaln Diek,Hellendoorn,7416,Buurt,BU01630408,1,810,405,405,135,85,200,210,180,330,410,35,35,40,50,0,0,0,5,45,NaN,NaN,NaN,NaN,330,70,125,135,2.5,2731.0,333,318.0,100.0,0.0,98.0,2.0,80.0,20.0,17.0,3.0,0.0,92.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,0.0,5.0,5.0,5.0,0.0,10.0,10.0,5.0,410,340,70,1.3,1389.0,45,0.8,0.8,0.9,NaN,NaN,31,30,1,7442,1.0,4.0,768.0,7.0,1.0,0.0,0.0,0.0,0.0
1,'s Gravenmoer,Dongen,4369,Buurt,BU07660216,1,2245,1175,1070,290,240,430,750,530,860,1095,175,115,160,65,0,10,5,5,40,NaN,NaN,NaN,NaN,995,260,390,340,2.3,1001.0,983,349.0,92.0,8.0,99.0,1.0,72.0,28.0,23.0,5.0,0.0,82.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320,10.0,50.0,50.0,25.0,40.0,75.0,35.0,30.0,1440,1165,275,1.5,642.0,150,3.6,0.6,0.5,NaN,NaN,226,224,2,5109,1.0,5.0,269.0,16.0,3.0,0.0,0.0,0.0,0.0
2,'s-Graveland,Wijdemeren,17141,Buurt,BU16960000,1,1330,655,675,150,135,245,470,335,630,470,160,75,150,70,10,10,

---
# 2021

### Load DataFrames

- 2021 - CBS Kerncijfers  - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=85039NED
- 2021 - CBS Voorzieningen   - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=85231NED

---

In [31]:
def rename_voorzieningen(df: pd.DataFrame) -> pd.DataFrame:

    return df.rename(columns={"Binnen1Km": "Huisarts_Binnen_1km", 
                              "Binnen3Km": "Huisarts_Binnen_3km", 
                              "Binnen5Km": "Huisarts_Binnen_5km",
                              "Binnen5Km.1": "Ziekenhuis_Binnen_5km",
                              "Binnen10Km": "Ziekenhuis_Binnen_10km",
                              "Binnen20Km": "Ziekenhuis_Binnen_20km",
                              "Binnen5Km.2": "Ziekenhuis_SEH_Binnen_5km",
                              "Binnen10Km.1": "Ziekenhuis_SEH_Binnen_10km",
                              "Binnen20Km.1": "Ziekenhuis_SEH_Binnen_20km",
                              "Binnen1Km.1": "Fysio_Binnen_1km",
                              "Binnen3Km.1": "Fysio_Binnen_3km",
                              "Binnen5Km.3": "Fysio_Binnen_5km",
                              "Binnen1Km.2": "Supermarkt_Binnen_1km",
                              "Binnen3Km.2": "Supermarkt_Binnen_3km",
                              "Binnen5Km.4": "Supermarkt_Binnen_5km",
                              "Binnen1Km.3": "Drogist_Binnen_1km",
                              "Binnen3Km.3": "Drogist_Binnen_3km",
                              "Binnen5Km.5": "Drogist_Binnen_5km",
                              "Binnen5Km.6": "Warenhuis_Binnen_5km",
                              "Binnen10Km.2": "Warenhuis_Binnen_10km",
                              "Binnen20Km.2": "Warenhuis_Binnen_20km",
                              "Binnen1Km.4": "Cafe_Binnen_1km",
                              "Binnen3Km.4": "Cafe_Binnen_3km",
                              "Binnen5Km.7": "Cafe_Binnen_5km",
                              "Binnen1Km.5": "Cafetaria_Binnen_1km",
                              "Binnen3Km.5": "Cafetaria_Binnen_3km",
                              "Binnen5Km.8": "Cafetaria_Binnen_5km",
                              "Binnen1Km.6": "Restaurant_Binnen_1km",
                              "Binnen3Km.6": "Restaurant_Binnen_3km",
                              "Binnen5Km.9": "Restaurant_Binnen_5km",
                              "Binnen5Km.10": "Hotel_Binnen_5km",
                              "Binnen10Km.3": "Hotel_Binnen_10km",
                              "Binnen20Km.3": "Hotel_Binnen_20km",
                              "Binnen1Km.7": "Kinderdagverblijf_Binnen_1km",
                              "Binnen3Km.7": "Kinderdagverblijf_Binnen_3km",
                              "Binnen5Km.11": "Kinderdagverblijf_Binnen_5km",
                              "Binnen1Km.8": "B_Opvang_Binnen_1km",
                              "Binnen3Km.8": "B_Opvang_Binnen_3km",
                              "Binnen5Km.12": "B_Opvang_Binnen_5km",
                              "Binnen1Km.9": "Basisschool_Binnen_1km",
                              "Binnen3Km.9": "Basisschool_Binnen_3km",
                              "Binnen5Km.13": "Basisschool_Binnen_5km",
                              "Binnen3Km.10": "VoortgezetOnderwijs_Binnen_3km",
                              "Binnen5Km.14": "VoortgezetOnderwijs_Binnen_5km",
                              "Binnen10Km.4": "VoortgezetOnderwijs_Binnen_10km",
                              "Binnen3Km.11": "VMBO_Binnen_3km",
                              "Binnen5Km.15": "VMBO_Binnen_5km",
                              "Binnen10Km.5": "VMBO_Binnen_10km",
                              "Binnen3Km.12": "HAVO_VWO_Binnen_3km",
                              "Binnen5Km.16": "HAVO_VWO_Binnen_5km",
                              "Binnen10Km.6": "HAVO_VWO_Binnen_10km",
                              "Binnen5Km.17": "Museum_Binnen_5km",
                              "Binnen10Km.7": "Museum_Binnen_10km",
                              "Binnen20Km.4": "Museum_Binnen_20km",
                              "Binnen5Km.18": "KunstPodium_Binnen_5km",
                              "Binnen10Km.8": "KunstPodium_Binnen_10km",
                              "Binnen20Km.5": "KunstPodium_Binnen_20km",
                              "Binnen5Km.19": "Bioscoop_Binnen_5km",
                              "Binnen10Km.9": "Bioscoop_Binnen_10km",
                              "Binnen20Km.6": "Bioscoop_Binnen_20km",
                              "Binnen10Km.10": "Attractie_Binnen_10km",
                              "Binnen20Km.7": "Attractie_Binnen_20km",
                              "Binnen50Km": "Attractie_Binnen_50km",
                              })

In [32]:
#====================================== Read & Clean the 2021 Datasets ============================================
kerncijfers_buurt_2021 =  rename_c_kerncijfers(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2021/CBS Buurtkerncijfers.gzip")))
voorzieningen_2021 = rename_voorzieningen(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2021/CBS Voorzieningen.gzip")))
#==================================================================================================================

#============================================ Name 2021 Datasets ==================================================
kerncijfers_buurt_2021.columns.name = "Kerncijfers 2021"
voorzieningen_2021.columns.name = "Voorzieningen 2021"
#==================================================================================================================

In [33]:
voorzieningen_2021

Voorzieningen 2021,ID,WijkenEnBuurten,Gemeentenaam,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AfstandTotHuisartsenpraktijk,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,AfstandTotHuisartsenpost,AfstandTotApotheek,AfstandTotZiekenhuis,Ziekenhuis_Binnen_5km,Ziekenhuis_Binnen_10km,Ziekenhuis_Binnen_20km,AfstandTotZiekenhuis.1,Ziekenhuis_SEH_Binnen_5km,Ziekenhuis_SEH_Binnen_10km,Ziekenhuis_SEH_Binnen_20km,AfstandTotConsultatiebureau,AfstandTotFysiotherapeut,Fysio_Binnen_1km,Fysio_Binnen_3km,Fysio_Binnen_5km,AfstandTotGroteSupermarkt,Supermarkt_Binnen_1km,Supermarkt_Binnen_3km,Supermarkt_Binnen_5km,AfstandTotOvDagelLevensmiddelen,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,AfstandTotWarenhuis,Warenhuis_Binnen_5km,Warenhuis_Binnen_10km,Warenhuis_Binnen_20km,AfstandTotCafeED,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,AfstandTotCafetariaED,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,AfstandTotRestaurant,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,AfstandTotHotelED,Hotel_Binnen_5km,Hotel_Binnen_10km,Hotel_Binnen_20km,AfstandTotKinderdagverblijf,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,AfstandTotBuitenschoolseOpvang,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,AfstandTotSchool,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,AfstandTotSchool.1,VoortgezetOnderwijs_Binnen_3km,VoortgezetOnderwijs_Binnen_5km,VoortgezetOnderwijs_Binnen_10km,AfstandTotSchool.2,VMBO_Binnen_3km,VMBO_Binnen_5km,VMBO_Binnen_10km,AfstandTotSchool.3,HAVO_VWO_Binnen_3km,HAVO_VWO_Binnen_5km,HAVO_VWO_Binnen_10km,AfstandTotOpenbaarGroenTotaal,AfstandTotParkOfPlantsoen,AfstandTotDagrecreatiefTerrein,AfstandTotBos,AfstandTotOpenNatTerreinTotaal,AfstandTotOpenDroogNatTerrein,AfstandTotOpenNatNatuurlijkTerrein,AfstandTotSemiOpenbaarGroenTotaal,AfstandTotSportterrein,AfstandTotVolkstuin,AfstandTotVerblijfsrecreatiefTerrein,AfstandTotBegraafplaats,AfstandTotRecreatiefBinnenwater,AfstandTotOpritHoofdverkeersweg,AfstandTotTreinstationsTotaal,AfstandTotBelangrijkOverstapstation,AfstandTotBibliotheek,AfstandTotZwembad,AfstandTotKunstijsbaan,AfstandTotMuseum,Museum_Binnen_5km,Museum_Binnen_10km,Museum_Binnen_20km,AfstandTotPodiumkunstenTotaal,KunstPodium_Binnen_5km,KunstPodium_Binnen_10km,KunstPodium_Binnen_20km,AfstandTotPoppodium,AfstandTotBioscoop,Bioscoop_Binnen_5km,Bioscoop_Binnen_10km,Bioscoop_Binnen_20km,AfstandTotSauna,AfstandTotZonnebank,AfstandTotAttractie,Attractie_Binnen_10km,Attractie_Binnen_20km,Attractie_Binnen_50km,AfstandTotBrandweerkazerne
0,3,Annen,Aa en Hunze,Buurt,BU16800000,1,0.8,0.8,1.0,2.9,15.1,0.8,13.0,0.0,0.0,4.2,15.1,0.0,0.0,1.2,NaN,NaN,NaN,NaN,NaN,0.8,1.3,2.5,4.3,0.8,2.8,4.5,9.7,5.1,0.4,2.0,10.0,4.1,0.0,0.0,2.9,4.6,0.0,0.0,4.6,0.7,2.8,6.4,15.3,2.7,4.0,12.0,33.4,0.5,2.5,5.9,8.1,0.6,2.0,4.8,8.0,0.8,0.7,2.6,4.7,7.3,0.0,0.0,1.0,7.3,0.0,0.0,1.0,13.7,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,13.3,14.8,0.7,5.1,25.1,5.7,0.1,1.0,8.0,14.0,0.0,0.0,3.0,21.6,14.3,0.0,0.0,2.1,13.9,13.6,5.5,1.0,7.0,26.0,1.3
1,4,Verspreide huizen Annen,Aa en Hunze,Buurt,BU16800009,1,2.6,0.0,0.7,1.3,16.2,2.6,12.1,0.0,0.1,4.8,16.2,0.0,0.0,1.9,NaN,NaN,NaN,NaN,NaN,2.1,0.1,1.8,3.0,1.9,0.1,3.5,5.1,6.3,0.0,2.0,9.0,5.4,0.0,0.0,0.4,5.8,0.0,0.0,0.7,1.6,0.2,4.5,9.6,2.8,3.3,10.5,32.9,1.7,0.1,3.3,7.5,1.8,0.1,2.8,7.0,2.0,0.0,1.2,3.6,7.0,0.0,0.1,1.0,7.0,0.0,0.1,1.0,12.8,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,13.6,16.0,2.1,6.8,26.8,7.2,0.0,1.0,8.0,13.2,0.0,0.0,3.3,23.3,14.0,0.0,0.0,2.4,13.4,12.7,7.0,1.0,7.0,25.7,1.7
2,6,Eext,Aa en Hunze,Buurt,BU16800100,1,3.8,0.0,0.0,2.0,14.4,3.9,14.4,0.0,0.0,4.0,14.4,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,0.6,0.8,1.0,3.2,4.0,0.0,0.0,6.8,4.2,1.0,1.1,8.0,3.5,0.0,0.0,2.0,2.2,0.0,1.0,4.0,0.5,1.9,3.0,7.8,0.6,6.0,7.6,31.4,1.2,0.3,1.0,4.9,1.2,0.3,1.0,6.3,0.9,0.7,1.0,4.3,3.9,0.0,1.0,1.0,3.9,0.0,1.0,1.0,15.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Remove Feature due to irrelevance

In [34]:
#=========================== Drop Duplicate Columns between Voorzieningen en Kerncijfers ==========================
voorzieningen_2021 = voorzieningen_2021.dropna(axis=1, how='all')
similar_columns = [c for c in voorzieningen_2021 if c in kerncijfers_buurt_2021]
similar_columns.remove("Codering")
voorzieningen_2021.drop(similar_columns, axis=1, inplace=True)
#==================================================================================================================

/var/folders/nx/t23l2x7x65l5f929y7y2ghlw0000gn/T/ipykernel_272/1692721893.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voorzieningen_2021.drop(similar_columns, axis=1, inplace=True)


In [35]:
voorzieningen_2021

Voorzieningen 2021,Codering,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,AfstandTotHuisartsenpost,AfstandTotApotheek,AfstandTotZiekenhuis,Ziekenhuis_Binnen_5km,Ziekenhuis_Binnen_10km,Ziekenhuis_Binnen_20km,AfstandTotZiekenhuis.1,Ziekenhuis_SEH_Binnen_5km,Ziekenhuis_SEH_Binnen_10km,Ziekenhuis_SEH_Binnen_20km,Supermarkt_Binnen_1km,Supermarkt_Binnen_3km,Supermarkt_Binnen_5km,AfstandTotOvDagelLevensmiddelen,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,AfstandTotWarenhuis,Warenhuis_Binnen_5km,Warenhuis_Binnen_10km,Warenhuis_Binnen_20km,AfstandTotCafeED,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,AfstandTotCafetariaED,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,AfstandTotRestaurant,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,AfstandTotHotelED,Hotel_Binnen_5km,Hotel_Binnen_10km,Hotel_Binnen_20km,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,AfstandTotBuitenschoolseOpvang,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,AfstandTotSchool.1,VoortgezetOnderwijs_Binnen_3km,VoortgezetOnderwijs_Binnen_5km,VoortgezetOnderwijs_Binnen_10km,AfstandTotSchool.2,VMBO_Binnen_3km,VMBO_Binnen_5km,VMBO_Binnen_10km,AfstandTotSchool.3,HAVO_VWO_Binnen_3km,HAVO_VWO_Binnen_5km,HAVO_VWO_Binnen_10km,AfstandTotOpritHoofdverkeersweg,AfstandTotTreinstationsTotaal,AfstandTotBelangrijkOverstapstation,AfstandTotBibliotheek,AfstandTotZwembad,AfstandTotKunstijsbaan,AfstandTotMuseum,Museum_Binnen_5km,Museum_Binnen_10km,Museum_Binnen_20km,AfstandTotPodiumkunstenTotaal,KunstPodium_Binnen_5km,KunstPodium_Binnen_10km,KunstPodium_Binnen_20km,AfstandTotPoppodium,AfstandTotBioscoop,Bioscoop_Binnen_5km,Bioscoop_Binnen_10km,Bioscoop_Binnen_20km,AfstandTotSauna,AfstandTotZonnebank,AfstandTotAttractie,Attractie_Binnen_10km,Attractie_Binnen_20km,Attractie_Binnen_50km,AfstandTotBrandweerkazerne
0,BU16800000,0.8,1.0,2.9,15.1,0.8,13.0,0.0,0.0,4.2,15.1,0.0,0.0,1.2,1.3,2.5,4.3,0.8,2.8,4.5,9.7,5.1,0.4,2.0,10.0,4.1,0.0,0.0,2.9,4.6,0.0,0.0,4.6,0.7,2.8,6.4,15.3,2.7,4.0,12.0,33.4,2.5,5.9,8.1,0.6,2.0,4.8,8.0,0.7,2.6,4.7,7.3,0.0,0.0,1.0,7.3,0.0,0.0,1.0,13.7,0.0,0.0,0.0,1.5,13.3,14.8,0.7,5.1,25.1,5.7,0.1,1.0,8.0,14.0,0.0,0.0,3.0,21.6,14.3,0.0,0.0,2.1,13.9,13.6,5.5,1.0,7.0,26.0,1.3
1,BU16800009,0.0,0.7,1.3,16.2,2.6,12.1,0.0,0.1,4.8,16.2,0.0,0.0,1.9,0.1,1.8,3.0,1.9,0.1,3.5,5.1,6.3,0.0,2.0,9.0,5.4,0.0,0.0,0.4,5.8,0.0,0.0,0.7,1.6,0.2,4.5,9.6,2.8,3.3,10.5,32.9,0.1,3.3,7.5,1.8,0.1,2.8,7.0,0.0,1.2,3.6,7.0,0.0,0.1,1.0,7.0,0.0,0.1,1.0,12.8,0.0,0.0,0.0,2.6,13.6,16.0,2.1,6.8,26.8,7.2,0.0,1.0,8.0,13.2,0.0,0.0,3.3,23.3,14.0,0.0,0.0,2.4,13.4,12.7,7.0,1.0,7.0,25.7,1.7
2,BU16800100,0.0,0.0,2.0,14.4,3.9,14.4,0.0,0.0,4.0,14.4,0.0,0.0,2.0,0.8,1.0,3.2,4.0,0.0,0.0,6.8,4.2,1.0,1.1,8.0,3.5,0.0,0.0,2.0,2.2,0.0,1.0,4.0,0.5,1.9,3.0,7.8,0.6,6.0,7.6,31.4,0.3,1.0,4.9,1.2,0.3,1.0,6.3,0.7,1.0,4.3,3.9,0.0,1.0,1.0,3.9,0.0,1.0,1.0,15.0,0.0,0.0,0.0,4.0,14.2,14.2,3.9,10.0,30.4,10.8,0.0,0.0,6.8,14.5,0.0,0.0,4.0,26.9,14.5,0.0,0.0,3.0,10.4,14.0,9.8,0.8,6.1,26.6,4.3
3,BU16800109,0.0,0.4,2.2,16.2,3.7,13.9,0.0,0.0,4.0,16.2,0.0,0.0,2.0,0.0,1.1,3.7,3.2,0.0,1.7,7.2,4.5,0.6,1.8,8.2,3.8,0.0,0.4,1.6,3.7,0.0,0.6,2.9,2.2,0.0,3.1,9.5,1.4,4.4,9.6,32.5,0.0,1.7,6.1,2.0,0.0,2.1,7.2,0.1,1.3,3.7,4.2,0.2,0.8,1.0,4.2,0.2,0.8,1.0,14.6,0.0,0.0,0.0,3.7,15.3,16.2,3.3,9.0,29.3,9.6,0.0,0.7,7.6,14.9,0.0,0.0,3.8,25.8,16.0,0.0,0.0,2.9,12.1,14.3,8.8,1.0,7.1,26.2,3.1
4,BU16800200,0.0,1.0,1.0,12.6,2.2,11.2,0.0,0.0,4.0,12.6,0.0,0.0,1.0,0.0,1.5,2.1,2.6,0.0,3.8,4.1,7.2,0.0,2.0,9.0,6.3,0.0,0.0,0.0,2.9,0.0,0.7,1.0,0.3,2.0,5.8,7.9,0.4,2.0,12.4,35.0,0.0,3.1,5.4,0.5,1.0,3.1,5.3,1.0,2.0,2.4,6.9,0.0,0.0,1.0,6.9,0.0,0.0,1.0,11.9,0.0,0.0,0.0,1.0,12.4,12.4,2.4,7.2,27.0,8.0,0.0,1.0,9.0,12.5,0.0,0.0,3.0,23.5,12.5,0.0,0.0,2.0,11.9,11.9,7.7,1.0,7.0,28.9,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

---
### Merge
---
- 2021 - Voorzieningen
- 2021 - Kerncijfers

In [36]:
#============================== Merge Voorzieningen 2021 met Kerncijfers 2021 =====================================
df_2021 = pd.merge(kerncijfers_buurt_2021, voorzieningen_2021, on="Codering", how="left").reset_index(drop=True)
df_2021.columns.name = "Dataset 2021"

#==================================================================================================================

---
### Merge
---
- 2022
- 2021

In [37]:
#======================================== Check Municipality Fusion =============================================== 
df_2021["Codering"] = df_2021["Codering"].apply(convert_codering, args=(2021, 2022))
matching_2021_2022 = df_2021[df_2021["Codering"].isin(df_2022["Codering"])].copy()
#==================================================================================================================

#========================================= Merge 2021 with 2022 ===================================================
df_2021_2022 = df_2022.set_index("Codering").combine_first(matching_2021_2022.set_index("Codering"))
df_2021_2022.columns.name = "Dataset 2021-2022"
contribution_dict[2021] = np.prod(df_2021_2022.shape) - df_2021_2022.isna().sum().sum() - contribution_dict[2022]
#==================================================================================================================

In [38]:
df_2021_2022

Dataset 2021-2022,ALandbouwBosbouwEnVisserij,AantalInkomensontvangers,AantalInwoners,AardgasVerbruikAppartement,AardgasVerbruikEigenWoning,AardgasVerbruikHoekwoning,AardgasVerbruikHuurwoning,AardgasVerbruikTussenwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AfstandTotApotheek,AfstandTotAttractie,AfstandTotBelangrijkOverstapstation,AfstandTotBibliotheek,AfstandTotBioscoop,AfstandTotBrandweerkazerne,AfstandTotBuitenschoolseOpvang,AfstandTotCafeED,AfstandTotCafetariaED,AfstandTotGroteSupermarkt,AfstandTotHotelED,AfstandTotHuisartsenpost,AfstandTotHuisartsenpraktijk,AfstandTotKinderdagverblijf,AfstandTotKunstijsbaan,AfstandTotMuseum,AfstandTotOpritHoofdverkeersweg,AfstandTotOvDagelLevensmiddelen,AfstandTotPodiumkunstenTotaal,AfstandTotPoppodium,AfstandTotRestaurant,AfstandTotSauna,AfstandTotSchool,AfstandTotSchool.1,AfstandTotSchool.2,AfstandTotSchool.3,AfstandTotTreinstationsTotaal,AfstandTotWarenhuis,AfstandTotZiekenhuis,AfstandTotZiekenhuis.1,AfstandTotZonnebank,AfstandTotZwembad,Attractie_Binnen_10km,Attractie_Binnen_20km,Attractie_Binnen_50km,BFNijverheidEnEnergie,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,BedrijfsvestigingenTotaal,Berovingen,Bevolkingsdichtheid,Bioscoop_Binnen_10km,Bioscoop_Binnen_20km,Bioscoop_Binnen_5km,BouwjaarVanaf2000,BouwjaarVoor2000,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,Dekkingspercentage,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,Eenpersoonshuishoudens,EigendomOnbekend,EnergieVerbruikAppartement,EnergieVerbruikEigenWoning,EnergieVerbruikHoekwoning,EnergieVerbruikHuurwoning,EnergieVerbruikTussenwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,GIHandelEnHoreca,GeboorteRelatief,GeboorteTotaal,Gehuwd,GemGestandaardiseerdInkomenVanHuish,Gemeentenaam,GemiddeldAardgasverbruikTotaal,GemiddeldElektriciteitsverbruikTotaal,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,GemiddeldeHuishoudensgrootte,GemiddeldeWOZWaardeVanWoningen,GeregistreerdeOverlast,Gescheiden,Geweld,HAVO_VWO_Binnen_10km,HAVO_VWO_Binnen_3km,HAVO_VWO_Binnen_5km,HJVervoerInformatieEnCommunicatie,Hotel_Binnen_10km,Hotel_Binnen_20km,Hotel_Binnen_5km,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,HuishOnderOfRondSociaalMinimum,HuishoudensMetEenLaagInkomen,HuishoudensMetKinderen,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,HuishoudensTotaal,HuishoudensZonderKinderen,HuurwoningenTotaal,ID,InBezitOverigeVerhuurders,InBezitWoningcorporatie,Inbraken,IndelingswijzigingWijkenEnBuurten,JongerenMetJeugdzorgInNatura,KLFinancieleDienstenOnroerendGoed,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,Koopwoningen,KunstPodium_Binnen_10km,KunstPodium_Binnen_20km,KunstPodium_Binnen_5km,MNZakelijkeDienstverlening,Mannen,Marokko,MateVanStedelijkheid,MediaanVermogenVanParticuliereHuish,MeestVoorkomendePostcode,Motorfietsen,Museum_Binnen_10km,Museum_Binnen_20km,Museum_Binnen_5km,NederlandseAntillenEnAruba,Nettoarbeidsparticipatie,NietWestersTotaal,OQOverheidOnderwijsEnZorg,Omgevingsadressendichtheid,Ongehuwd,OpleidingsniveauHoog,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OppervlakteLand,OppervlakteTotaal,OppervlakteWater,OverigNietWesters,PercentageBewoond,PercentageEengezinswoning,PercentageJongerenMetJeugdzorg,PercentageMeergezinswoning,PercentageOnbewoond,PercentageWerknemers,PercentageWoningenMetStadsverwarming,PercentageZelfstandigen,PersonenPerSoortUitkeringAO,PersonenPerSoortUitkeringAOW,PersonenPerSoortUitkeringBijstand,PersonenPerSoortUitkeringWW,PersonenautoSBrandstofBenzine,PersonenautoSNaarOppervlakte,PersonenautoSOverigeBrandstof,PersonenautoSPerHuishouden,PersonenautoSTotaal,RUCultuurRecreatieOverigeDiensten,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,ScholenBinnen3Km,SoortRegio,SterfteRelatief,SterfteTotaal,

---
# 2020

### Load DataFrames

- 2020 - CBS Kerncijfers  - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=84799NED
- 2020 - CBS Voorzieningen   - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=84953NED
- 2020 - Leefbarometer - https://www.leefbaarometer.nl/page/Open%20data

---

In [39]:
#====================================== Read & Clean the 2020 Datasets ============================================
kerncijfers_buurt_2020 =  rename_c_kerncijfers(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2020/CBS Buurtkerncijfers.gzip")))
voorzieningen_2020 = rename_voorzieningen(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2020/CBS Voorzieningen.gzip")))
leefbarometer_2020 = clean_open_data(pd.read_csv("https://data.overheid.nl/sites/default/files/dataset/f37c3649-cc52-4e48-a864-870ae42807a2/resources/Leefbaarometer%203.0%20-%20meting%202020%20-%20scores%20buurt.csv"))
gezondheid_2020 = clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2020/CBS Gezondheid.gzip"))
zonnepanelen_2020 = clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2020/CBS Zonnepanelen.gzip"))
#==================================================================================================================

#============================================ Name 2020 Datasets ==================================================
kerncijfers_buurt_2020.columns.name = "Kerncijfers 2020"
voorzieningen_2020.columns.name = "Voorzieningen 2020"
leefbarometer_2020.columns.name = "Leefbarometer 2020"
gezondheid_2020.columns.name = "Gezondheid 2020"
zonnepanelen_2020.columns.name = "Zonnepanelen 2020"
#==================================================================================================================

In [40]:
zonnepanelen_2020

Zonnepanelen 2020,ID,WijkenEnBuurten,Gemeentenaam,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AantalInstallatiesBijWoningen,OpgesteldVermogenVanZonnepanelen
0,3,Annen,Aa en Hunze,Buurt,BU16800000,1,499.0,1884.0
1,4,Verspreide huizen Annen,Aa en Hunze,Buurt,BU16800009,1,12.0,49.0
2,6,Eext,Aa en Hunze,Buurt,BU16800100,1,169.0,652.0
3,7,Verspreide huizen Eext,Aa en Hunze,Buurt,BU16800109,1,17.0,87.0
4,9,Anloo,Aa en Hunze,Buurt,BU16800200,1,43.0,181.0
...,...,...,...,...,...,...,...,...
12804,17335,Gerenlanden,Zwolle,Buurt,BU01935140,1,392.0,1152.0
12805,17336,Gerenbroek,Zwolle,Buurt,BU01935150,1,352.0,1046.0
12806,17337,Oude Mars,Zwolle,Buurt,BU01935160,1,19.0,101.0
12807,17339,Windesheim,Zwolle,Buurt,BU01935200,1,45.0,186.0


In [41]:
show_info([zonnepanelen_2020])

Info Dataset Zonnepanelen 2020
- Total Values:                         102472
- Total Missings:                       2428
- % Missing:                            2.37%
- Dataset Dimensions:                   (12809, 8)



In [42]:
#============================ Filter op jaar en get unique columns LBM 2020 Datasets ==============================
leefbarometer_2020 = leefbarometer_2020[leefbarometer_2020["jaar"] == 2020].rename(columns={'bucode': 'Codering'})
columns_to_use = leefbarometer_2020.columns.difference(kerncijfers_buurt_2020.columns).tolist()
#==================================================================================================================

#============================ Remove unwanted columns LBM 2020 Datasets & add codering ============================
columns_to_use.remove("bunaam")
columns_to_use.remove("jaar")
columns_to_use.remove("versie")
columns_to_use.append("Codering")
#==================================================================================================================

#=================================== Merge LBM 2020 Dataset met Kerncijfers 2020===================================
leefbarometer_2020 = leefbarometer_2020[columns_to_use]
kerncijfers_buurt_2020 = pd.merge(kerncijfers_buurt_2020, leefbarometer_2020, on="Codering", how="left").reset_index(drop=True)
kerncijfers_buurt_2020.columns.name = "Dataset 2020"
#==================================================================================================================

In [43]:
#=========================== Drop Duplicate Columns between Voorzieningen en Kerncijfers ==========================
voorzieningen_2020 = voorzieningen_2020.dropna(axis=1, how='all')
similar_columns = [c for c in voorzieningen_2020 if c in kerncijfers_buurt_2020]
similar_columns.remove("Codering")
voorzieningen_2020.drop(similar_columns, axis=1, inplace=True)
#==================================================================================================================

---
### Merge
---
- 2020 - Voorzieningen
- 2020 - Kerncijfers

In [44]:
#============================== Merge Voorzieningen 2020 met Kerncijfers 2020 =====================================
df_2020 = pd.merge(kerncijfers_buurt_2020, voorzieningen_2020, on="Codering", how="left").reset_index(drop=True)
df_2020.columns.name = "Dataset 2020"
#==================================================================================================================

---
### Merge
---
- 2020 - Zonnepanelen
- 2020 - Kerncijfers

In [45]:
df_2020

Dataset 2020,ID,WijkenEnBuurten,Gemeentenaam,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AantalInwoners,Mannen,Vrouwen,k0Tot15Jaar,k15Tot25Jaar,k25Tot45Jaar,k45Tot65Jaar,k65JaarOfOuder,Ongehuwd,Gehuwd,Gescheiden,Verweduwd,WestersTotaal,NietWestersTotaal,Marokko,NederlandseAntillenEnAruba,Suriname,Turkije,OverigNietWesters,GeboorteTotaal,GeboorteRelatief,SterfteTotaal,SterfteRelatief,HuishoudensTotaal,Eenpersoonshuishoudens,HuishoudensZonderKinderen,HuishoudensMetKinderen,GemiddeldeHuishoudensgrootte,Bevolkingsdichtheid,Woningvoorraad,GemiddeldeWOZWaardeVanWoningen,PercentageEengezinswoning,PercentageMeergezinswoning,PercentageBewoond,PercentageOnbewoond,Koopwoningen,HuurwoningenTotaal,InBezitWoningcorporatie,InBezitOverigeVerhuurders,EigendomOnbekend,BouwjaarVoor2000,BouwjaarVanaf2000,GemiddeldElektriciteitsverbruikTotaal,EnergieVerbruikAppartement,EnergieVerbruikTussenwoning,EnergieVerbruikHoekwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,EnergieVerbruikHuurwoning,EnergieVerbruikEigenWoning,GemiddeldAardgasverbruikTotaal,AardgasVerbruikAppartement,AardgasVerbruikTussenwoning,AardgasVerbruikHoekwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AardgasVerbruikHuurwoning,AardgasVerbruikEigenWoning,PercentageWoningenMetStadsverwarming,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OpleidingsniveauHoog,Nettoarbeidsparticipatie,PercentageWerknemers,PercentageZelfstandigen,AantalInkomensontvangers,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,k40PersonenMetLaagsteInkomen,k20PersonenMetHoogsteInkomen,GemGestandaardiseerdInkomenVanHuish,k40HuishoudensMetLaagsteInkomen,k20HuishoudensMetHoogsteInkomen,HuishoudensMetEenLaagInkomen,HuishOnderOfRondSociaalMinimum,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,MediaanVermogenVanParticuliereHuish,PersonenPerSoortUitkeringBijstand,PersonenPerSoortUitkeringAO,PersonenPerSoortUitkeringWW,PersonenPerSoortUitkeringAOW,JongerenMetJeugdzorgInNatura,PercentageJongerenMetJeugdzorg,WmoClienten,WmoClientenRelatief,BedrijfsvestigingenTotaal,ALandbouwBosbouwEnVisserij,BFNijverheidEnEnergie,GIHandelEnHoreca,HJVervoerInformatieEnCommunicatie,KLFinancieleDienstenOnroerendGoed,MNZakelijkeDienstverlening,OQOverheidOnderwijsEnZorg,RUCultuurRecreatieOverigeDiensten,PersonenautoSTotaal,PersonenautoSBrandstofBenzine,PersonenautoSOverigeBrandstof,PersonenautoSPerHuishouden,PersonenautoSNaarOppervlakte,Motorfietsen,AfstandTotHuisartsenpraktijk,AfstandTotGroteSupermarkt,AfstandTotKinderdagverblijf,AfstandTotSchool,ScholenBinnen3Km,OppervlakteTotaal,OppervlakteLand,OppervlakteWater,MeestVoorkomendePostcode,Dekkingspercentage,MateVanStedelijkheid,Omgevingsadressendichtheid,afw,fys,lbm,onv,soc,vrz,won,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,AfstandTotHuisartsenpost,AfstandTotApotheek,AfstandTotZiekenhuis,Ziekenhuis_Binnen_5km,Ziekenhuis_Binnen_10km,Ziekenhuis_Binnen_20km,AfstandTotZiekenhuis.1,Ziekenhuis_SEH_Binnen_5km,Ziekenhuis_SEH_Binnen_10km,Ziekenhuis_SEH_Binnen_20km,Supermarkt_Binnen_1km,Supermarkt_Binnen_3km,Supermarkt_Binnen_5km,AfstandTotOvDagelLevensmiddelen,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,AfstandTotWarenhuis,Warenhuis_Binnen_5km,Warenhuis_Binnen_10km,Warenhuis_Binnen_20km,AfstandTotCafeED,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,AfstandTotCafetariaED,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,AfstandTotRestaurant,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,AfstandTotHotelED,Hotel_Binnen_5km,Hotel_Binnen_10km,Hotel_Binnen_20km,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,AfstandTotBuitenschoolseOpvang,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,AfstandTotSchool.1,VoortgezetOnderwijs_Binnen_3km,VoortgezetOnderwijs_Binnen_5km,VoortgezetOnderwijs_Binnen_10km,AfstandTotSchool.2,VMBO_Binne

In [46]:
#=========================== Drop Duplicate Columns between Zonnepanelen en Kerncijfers ==========================
zonnepanelen_2020 = zonnepanelen_2020.dropna(axis=1, how='all')
similar_columns = [c for c in zonnepanelen_2020 if c in df_2020]
similar_columns.remove("Codering")
zonnepanelen_2020.drop(similar_columns, axis=1, inplace=True)
#==================================================================================================================

In [47]:
#============================== Merge Zonnepanelen 2020 met Kerncijfers 2020 =====================================
df_2020 = pd.merge(df_2020, zonnepanelen_2020, on="Codering", how="left").reset_index(drop=True)
df_2020.columns.name = "Dataset 2020"
#==================================================================================================================

In [48]:
df_2020

Dataset 2020,ID,WijkenEnBuurten,Gemeentenaam,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AantalInwoners,Mannen,Vrouwen,k0Tot15Jaar,k15Tot25Jaar,k25Tot45Jaar,k45Tot65Jaar,k65JaarOfOuder,Ongehuwd,Gehuwd,Gescheiden,Verweduwd,WestersTotaal,NietWestersTotaal,Marokko,NederlandseAntillenEnAruba,Suriname,Turkije,OverigNietWesters,GeboorteTotaal,GeboorteRelatief,SterfteTotaal,SterfteRelatief,HuishoudensTotaal,Eenpersoonshuishoudens,HuishoudensZonderKinderen,HuishoudensMetKinderen,GemiddeldeHuishoudensgrootte,Bevolkingsdichtheid,Woningvoorraad,GemiddeldeWOZWaardeVanWoningen,PercentageEengezinswoning,PercentageMeergezinswoning,PercentageBewoond,PercentageOnbewoond,Koopwoningen,HuurwoningenTotaal,InBezitWoningcorporatie,InBezitOverigeVerhuurders,EigendomOnbekend,BouwjaarVoor2000,BouwjaarVanaf2000,GemiddeldElektriciteitsverbruikTotaal,EnergieVerbruikAppartement,EnergieVerbruikTussenwoning,EnergieVerbruikHoekwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,EnergieVerbruikHuurwoning,EnergieVerbruikEigenWoning,GemiddeldAardgasverbruikTotaal,AardgasVerbruikAppartement,AardgasVerbruikTussenwoning,AardgasVerbruikHoekwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AardgasVerbruikHuurwoning,AardgasVerbruikEigenWoning,PercentageWoningenMetStadsverwarming,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OpleidingsniveauHoog,Nettoarbeidsparticipatie,PercentageWerknemers,PercentageZelfstandigen,AantalInkomensontvangers,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,k40PersonenMetLaagsteInkomen,k20PersonenMetHoogsteInkomen,GemGestandaardiseerdInkomenVanHuish,k40HuishoudensMetLaagsteInkomen,k20HuishoudensMetHoogsteInkomen,HuishoudensMetEenLaagInkomen,HuishOnderOfRondSociaalMinimum,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,MediaanVermogenVanParticuliereHuish,PersonenPerSoortUitkeringBijstand,PersonenPerSoortUitkeringAO,PersonenPerSoortUitkeringWW,PersonenPerSoortUitkeringAOW,JongerenMetJeugdzorgInNatura,PercentageJongerenMetJeugdzorg,WmoClienten,WmoClientenRelatief,BedrijfsvestigingenTotaal,ALandbouwBosbouwEnVisserij,BFNijverheidEnEnergie,GIHandelEnHoreca,HJVervoerInformatieEnCommunicatie,KLFinancieleDienstenOnroerendGoed,MNZakelijkeDienstverlening,OQOverheidOnderwijsEnZorg,RUCultuurRecreatieOverigeDiensten,PersonenautoSTotaal,PersonenautoSBrandstofBenzine,PersonenautoSOverigeBrandstof,PersonenautoSPerHuishouden,PersonenautoSNaarOppervlakte,Motorfietsen,AfstandTotHuisartsenpraktijk,AfstandTotGroteSupermarkt,AfstandTotKinderdagverblijf,AfstandTotSchool,ScholenBinnen3Km,OppervlakteTotaal,OppervlakteLand,OppervlakteWater,MeestVoorkomendePostcode,Dekkingspercentage,MateVanStedelijkheid,Omgevingsadressendichtheid,afw,fys,lbm,onv,soc,vrz,won,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,AfstandTotHuisartsenpost,AfstandTotApotheek,AfstandTotZiekenhuis,Ziekenhuis_Binnen_5km,Ziekenhuis_Binnen_10km,Ziekenhuis_Binnen_20km,AfstandTotZiekenhuis.1,Ziekenhuis_SEH_Binnen_5km,Ziekenhuis_SEH_Binnen_10km,Ziekenhuis_SEH_Binnen_20km,Supermarkt_Binnen_1km,Supermarkt_Binnen_3km,Supermarkt_Binnen_5km,AfstandTotOvDagelLevensmiddelen,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,AfstandTotWarenhuis,Warenhuis_Binnen_5km,Warenhuis_Binnen_10km,Warenhuis_Binnen_20km,AfstandTotCafeED,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,AfstandTotCafetariaED,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,AfstandTotRestaurant,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,AfstandTotHotelED,Hotel_Binnen_5km,Hotel_Binnen_10km,Hotel_Binnen_20km,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,AfstandTotBuitenschoolseOpvang,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,AfstandTotSchool.1,VoortgezetOnderwijs_Binnen_3km,VoortgezetOnderwijs_Binnen_5km,VoortgezetOnderwijs_Binnen_10km,AfstandTotSchool.2,VMBO_Binne

---
### Merge
---
- 2020 - Gezondheid
- 2020 - Kerncijfers

In [49]:
gezondheid_2020

Gezondheid 2020,ID,Leeftijd,Marges,WijkenEnBuurten,Perioden,Gemeentenaam,SoortRegio,Codering,ErvarenGezondheidGoedZeerGoed,VoldoetAanBeweegrichtlijn,WekelijkseSporters,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,Roker,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker,EenOfMeerLangdurigeAandoeningen,BeperktVanwegeGezondheid,ErnstigBeperktVanwegeGezondheid,LangdurigeZiekteEnBeperkt,EenOfMeerLichamelijkeBeperkingen,BeperkingInHoren,BeperkingInZien,BeperkingInBewegen,MatigHoogRisicoOpAngstOfDepressie,HoogRisicoOpAngstOfDepressie,HeelVeelStressInAfgelopen4Weken,MatigVeelRegieOverEigenLeven,Eenzaam,ErnstigZeerErnstigEenzaam,EmotioneelEenzaam,SociaalEenzaam,Mantelzorger,Vrijwilligerswerk,LopenEnOfFietsenNaarSchoolOfWerk,LopenNaarSchoolOfWerk,FietsenNaarSchoolOfWerk,ErnstigeGeluidhinderDoorBuren,ErnstigeSlaapverstoringVliegverkeer,ErnstigeGeluidhinderDoorVliegverkeer,ErnstigeGeluidhinderDoorTreinverkeer,ErnstigeGeluidhinderWeg,ErnstigeGeluidhinderWeg50KmUur,ErnstigeGeluidhinderWeg50KmUur.1,MoeiteMetRondkomen,BrozeGezondheid,BrozeGezondheidPsychologischeDomein,BrozeFysiekeGezondheid,BrozeGezondheidSocialeDomein
0,11,18 jaar of ouder,Waarde,Annen,2020,Aa en Hunze,Buurt,BU16800000,81.3,47.5,47.8,1.6,46.4,53.2,15.3,13.6,40.0,80.7,7.0,5.8,34.4,31.8,4.7,30.3,13.6,4.0,3.9,9.0,32.8,3.7,12.2,92.5,41.9,7.6,21.9,26.7,14.4,29.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,NaN,NaN,NaN,NaN
1,14,18 jaar of ouder,Waarde,Verspreide huizen Annen,2020,Aa en Hunze,Buurt,BU16800009,84.4,50.0,48.3,1.6,51.9,49.1,12.4,12.5,42.2,85.0,7.2,7.4,32.3,26.9,3.2,25.3,8.9,3.2,2.8,4.7,27.3,2.3,9.9,94.2,39.3,5.5,17.9,26.3,17.1,35.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,NaN,NaN,NaN
2,20,18 jaar of ouder,Waarde,Eext,2020,Aa en Hunze,Buurt,BU16800100,81.8,49.2,45.4,1.8,44.9,54.4,15.4,15.7,39.3,81.4,8.3,7.2,31.6,30.3,4.3,28.7,11.8,3.9,3.3,6.9,34.7,4.1,13.2,92.8,41.6,7.8,21.8,28.7,14.7,31.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.5,NaN,NaN,NaN,NaN
3,23,18 jaar of ouder,Waarde,Verspreide huizen Eext,2020,Aa en Hunze,Buurt,BU16800109,82.1,49.0,43.8,1.7,47.4,52.7,14.3,13.1,42.4,82.2,7.2,8.3,34.3,32.3,4.7,30.7,11.5,4.1,3.4,6.6,31.4,3.3,12.0,92.5,41.7,7.5,20.7,29.8,14.8,32.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.3,NaN,NaN,NaN,NaN
4,29,18 jaar of ouder,Waarde,Anloo,2020,Aa en Hunze,Buurt,BU16800200,81.9,46.8,46.2,1.5,46.6,51.4,14.8,13.3,40.8,81.0,6.5,6.9,33.6,32.8,4.7,30.8,13.3,4.1,3.3,8.6,31.1,3.7,11.1,93.0,43.3,6.7,19.9,30.5,16.5,33.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41419,156053,18 jaar of ouder,Bovengrens 95%-interval,Gerenlanden,2020,Zwolle,Buurt,BU01935140,84.4,56.6,54.2,2.4,49.9,52.8,16.3,18.5,43.0,83.8,9.4,7.2,33.8,30.3,4.9,29.1,12.9,4.2,4.9,7.6,47.7,5.9,18.5,93.7,44.9,9.9,26.3,27.8,16.7,36.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6,NaN,NaN,NaN,NaN
41420,156056,18 jaar of ouder,Bovengrens 95%-interval,Gerenbroek,2020,Zwolle,Buurt,BU01935150,84.9,57.5,53.5,2.1,48.5,54.4,17.1,17.7,44.0,83.4,9.1,7.2,33.7,30.2,4.7,28.9,12.5,4.1,4.8,7.0,46.1,5.6,18.1,94.1,44.3,9.7,25.4,27.9,17.5,36.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.9,NaN,NaN,NaN,NaN
41421,156059,18 jaar of ouder,Bovengrens 95%-interval,Oude Mars,2020,Zwolle,Buurt,BU01935160,98.5,75.0,80.9,5.2,69.1,51.5,16.2,16.2,44.1,97.1,16.2,11.8,32.4,26.5,4.4,25.0,13.2,5.9,5.9,7.4,45.6,7.4,25.0,100.0,39.7,8.8,23.5,25.0,23.5,57.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,NaN,NaN,NaN,NaN
41422,156065,18 jaar of ouder,Bovengrens 95%-interval,Windesheim,2020,Zwolle,Buurt,BU01935200,87.9,55.7,51.1,3.3,53.4,56.4,17.9,19.5,43.0,87.3,14.3,11.4,33.9,31.6,5.2,30.3,13.7,5.2,5.5,7.8,46.6,6.2,16.6,96.3,44.3,9.4,25.1,29.0,18.6,47.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.4,NaN,NaN,NaN,NaN


In [50]:
df_2020

Dataset 2020,ID,WijkenEnBuurten,Gemeentenaam,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AantalInwoners,Mannen,Vrouwen,k0Tot15Jaar,k15Tot25Jaar,k25Tot45Jaar,k45Tot65Jaar,k65JaarOfOuder,Ongehuwd,Gehuwd,Gescheiden,Verweduwd,WestersTotaal,NietWestersTotaal,Marokko,NederlandseAntillenEnAruba,Suriname,Turkije,OverigNietWesters,GeboorteTotaal,GeboorteRelatief,SterfteTotaal,SterfteRelatief,HuishoudensTotaal,Eenpersoonshuishoudens,HuishoudensZonderKinderen,HuishoudensMetKinderen,GemiddeldeHuishoudensgrootte,Bevolkingsdichtheid,Woningvoorraad,GemiddeldeWOZWaardeVanWoningen,PercentageEengezinswoning,PercentageMeergezinswoning,PercentageBewoond,PercentageOnbewoond,Koopwoningen,HuurwoningenTotaal,InBezitWoningcorporatie,InBezitOverigeVerhuurders,EigendomOnbekend,BouwjaarVoor2000,BouwjaarVanaf2000,GemiddeldElektriciteitsverbruikTotaal,EnergieVerbruikAppartement,EnergieVerbruikTussenwoning,EnergieVerbruikHoekwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,EnergieVerbruikHuurwoning,EnergieVerbruikEigenWoning,GemiddeldAardgasverbruikTotaal,AardgasVerbruikAppartement,AardgasVerbruikTussenwoning,AardgasVerbruikHoekwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AardgasVerbruikHuurwoning,AardgasVerbruikEigenWoning,PercentageWoningenMetStadsverwarming,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OpleidingsniveauHoog,Nettoarbeidsparticipatie,PercentageWerknemers,PercentageZelfstandigen,AantalInkomensontvangers,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,k40PersonenMetLaagsteInkomen,k20PersonenMetHoogsteInkomen,GemGestandaardiseerdInkomenVanHuish,k40HuishoudensMetLaagsteInkomen,k20HuishoudensMetHoogsteInkomen,HuishoudensMetEenLaagInkomen,HuishOnderOfRondSociaalMinimum,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,MediaanVermogenVanParticuliereHuish,PersonenPerSoortUitkeringBijstand,PersonenPerSoortUitkeringAO,PersonenPerSoortUitkeringWW,PersonenPerSoortUitkeringAOW,JongerenMetJeugdzorgInNatura,PercentageJongerenMetJeugdzorg,WmoClienten,WmoClientenRelatief,BedrijfsvestigingenTotaal,ALandbouwBosbouwEnVisserij,BFNijverheidEnEnergie,GIHandelEnHoreca,HJVervoerInformatieEnCommunicatie,KLFinancieleDienstenOnroerendGoed,MNZakelijkeDienstverlening,OQOverheidOnderwijsEnZorg,RUCultuurRecreatieOverigeDiensten,PersonenautoSTotaal,PersonenautoSBrandstofBenzine,PersonenautoSOverigeBrandstof,PersonenautoSPerHuishouden,PersonenautoSNaarOppervlakte,Motorfietsen,AfstandTotHuisartsenpraktijk,AfstandTotGroteSupermarkt,AfstandTotKinderdagverblijf,AfstandTotSchool,ScholenBinnen3Km,OppervlakteTotaal,OppervlakteLand,OppervlakteWater,MeestVoorkomendePostcode,Dekkingspercentage,MateVanStedelijkheid,Omgevingsadressendichtheid,afw,fys,lbm,onv,soc,vrz,won,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,AfstandTotHuisartsenpost,AfstandTotApotheek,AfstandTotZiekenhuis,Ziekenhuis_Binnen_5km,Ziekenhuis_Binnen_10km,Ziekenhuis_Binnen_20km,AfstandTotZiekenhuis.1,Ziekenhuis_SEH_Binnen_5km,Ziekenhuis_SEH_Binnen_10km,Ziekenhuis_SEH_Binnen_20km,Supermarkt_Binnen_1km,Supermarkt_Binnen_3km,Supermarkt_Binnen_5km,AfstandTotOvDagelLevensmiddelen,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,AfstandTotWarenhuis,Warenhuis_Binnen_5km,Warenhuis_Binnen_10km,Warenhuis_Binnen_20km,AfstandTotCafeED,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,AfstandTotCafetariaED,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,AfstandTotRestaurant,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,AfstandTotHotelED,Hotel_Binnen_5km,Hotel_Binnen_10km,Hotel_Binnen_20km,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,AfstandTotBuitenschoolseOpvang,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,AfstandTotSchool.1,VoortgezetOnderwijs_Binnen_3km,VoortgezetOnderwijs_Binnen_5km,VoortgezetOnderwijs_Binnen_10km,AfstandTotSchool.2,VMBO_Binne

In [51]:
gezondheid_2020 = gezondheid_2020[gezondheid_2020["Marges"] == "Waarde"]
gezondheid_2020

Gezondheid 2020,ID,Leeftijd,Marges,WijkenEnBuurten,Perioden,Gemeentenaam,SoortRegio,Codering,ErvarenGezondheidGoedZeerGoed,VoldoetAanBeweegrichtlijn,WekelijkseSporters,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,Roker,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker,EenOfMeerLangdurigeAandoeningen,BeperktVanwegeGezondheid,ErnstigBeperktVanwegeGezondheid,LangdurigeZiekteEnBeperkt,EenOfMeerLichamelijkeBeperkingen,BeperkingInHoren,BeperkingInZien,BeperkingInBewegen,MatigHoogRisicoOpAngstOfDepressie,HoogRisicoOpAngstOfDepressie,HeelVeelStressInAfgelopen4Weken,MatigVeelRegieOverEigenLeven,Eenzaam,ErnstigZeerErnstigEenzaam,EmotioneelEenzaam,SociaalEenzaam,Mantelzorger,Vrijwilligerswerk,LopenEnOfFietsenNaarSchoolOfWerk,LopenNaarSchoolOfWerk,FietsenNaarSchoolOfWerk,ErnstigeGeluidhinderDoorBuren,ErnstigeSlaapverstoringVliegverkeer,ErnstigeGeluidhinderDoorVliegverkeer,ErnstigeGeluidhinderDoorTreinverkeer,ErnstigeGeluidhinderWeg,ErnstigeGeluidhinderWeg50KmUur,ErnstigeGeluidhinderWeg50KmUur.1,MoeiteMetRondkomen,BrozeGezondheid,BrozeGezondheidPsychologischeDomein,BrozeFysiekeGezondheid,BrozeGezondheidSocialeDomein
0,11,18 jaar of ouder,Waarde,Annen,2020,Aa en Hunze,Buurt,BU16800000,81.3,47.5,47.8,1.6,46.4,53.2,15.3,13.6,40.0,80.7,7.0,5.8,34.4,31.8,4.7,30.3,13.6,4.0,3.9,9.0,32.8,3.7,12.2,92.5,41.9,7.6,21.9,26.7,14.4,29.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,NaN,NaN,NaN,NaN
1,14,18 jaar of ouder,Waarde,Verspreide huizen Annen,2020,Aa en Hunze,Buurt,BU16800009,84.4,50.0,48.3,1.6,51.9,49.1,12.4,12.5,42.2,85.0,7.2,7.4,32.3,26.9,3.2,25.3,8.9,3.2,2.8,4.7,27.3,2.3,9.9,94.2,39.3,5.5,17.9,26.3,17.1,35.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,NaN,NaN,NaN
2,20,18 jaar of ouder,Waarde,Eext,2020,Aa en Hunze,Buurt,BU16800100,81.8,49.2,45.4,1.8,44.9,54.4,15.4,15.7,39.3,81.4,8.3,7.2,31.6,30.3,4.3,28.7,11.8,3.9,3.3,6.9,34.7,4.1,13.2,92.8,41.6,7.8,21.8,28.7,14.7,31.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.5,NaN,NaN,NaN,NaN
3,23,18 jaar of ouder,Waarde,Verspreide huizen Eext,2020,Aa en Hunze,Buurt,BU16800109,82.1,49.0,43.8,1.7,47.4,52.7,14.3,13.1,42.4,82.2,7.2,8.3,34.3,32.3,4.7,30.7,11.5,4.1,3.4,6.6,31.4,3.3,12.0,92.5,41.7,7.5,20.7,29.8,14.8,32.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.3,NaN,NaN,NaN,NaN
4,29,18 jaar of ouder,Waarde,Anloo,2020,Aa en Hunze,Buurt,BU16800200,81.9,46.8,46.2,1.5,46.6,51.4,14.8,13.3,40.8,81.0,6.5,6.9,33.6,32.8,4.7,30.8,13.3,4.1,3.3,8.6,31.1,3.7,11.1,93.0,43.3,6.7,19.9,30.5,16.5,33.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13803,52007,18 jaar of ouder,Waarde,Gerenlanden,2020,Zwolle,Buurt,BU01935140,82.5,54.9,52.2,1.9,48.3,51.0,14.9,16.4,40.8,81.9,8.2,6.1,31.9,28.7,4.1,27.6,11.8,3.5,4.3,6.6,45.5,4.7,17.0,92.5,42.8,8.6,24.9,26.1,15.3,34.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.6,NaN,NaN,NaN,NaN
13804,52010,18 jaar of ouder,Waarde,Gerenbroek,2020,Zwolle,Buurt,BU01935150,83.1,55.1,51.3,1.7,46.8,52.7,15.7,15.9,42.0,81.8,7.9,6.1,31.8,28.3,4.0,27.3,11.3,3.4,4.0,6.0,44.4,4.5,16.2,93.1,42.3,8.4,23.9,26.2,16.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.2,NaN,NaN,NaN,NaN
13805,52013,18 jaar of ouder,Waarde,Oude Mars,2020,Zwolle,Buurt,BU01935160,92.3,63.4,71.2,1.7,59.0,40.7,9.8,9.1,33.5,91.0,9.2,6.4,23.0,18.1,1.6,17.9,7.2,2.6,2.2,3.0,34.7,2.6,16.1,97.2,28.5,4.1,14.5,16.3,14.8,47.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN
13806,52019,18 jaar of ouder,Waarde,Windesheim,2020,Zwolle,Buurt,BU01935200,84.3,50.1,45.7,1.7,48.1,50.9,13.8,15.6,37.7,83.2,10.6,7.8,29.0,27.3,3.1,25.7,10.2,3.2,3.3,5.2,41.3,3.8,12.7,94.0,39.4,6.5,20.5,24.5,14.6,40.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.6,NaN,NaN,NaN,NaN


In [52]:
#=========================== Drop Duplicate Columns between gezondheid en Kerncijfers ==========================
gezondheid_2020 = gezondheid_2020.dropna(axis=1, how='all')
similar_columns = [c for c in gezondheid_2020 if c in df_2020]
similar_columns.remove("Codering")
gezondheid_2020.drop(similar_columns, axis=1, inplace=True)
#==================================================================================================================

/var/folders/nx/t23l2x7x65l5f929y7y2ghlw0000gn/T/ipykernel_272/3266785892.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gezondheid_2020.drop(similar_columns, axis=1, inplace=True)


In [53]:
gezondheid_2020

Gezondheid 2020,Leeftijd,Marges,Perioden,Codering,ErvarenGezondheidGoedZeerGoed,VoldoetAanBeweegrichtlijn,WekelijkseSporters,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,Roker,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker,EenOfMeerLangdurigeAandoeningen,BeperktVanwegeGezondheid,ErnstigBeperktVanwegeGezondheid,LangdurigeZiekteEnBeperkt,EenOfMeerLichamelijkeBeperkingen,BeperkingInHoren,BeperkingInZien,BeperkingInBewegen,MatigHoogRisicoOpAngstOfDepressie,HoogRisicoOpAngstOfDepressie,HeelVeelStressInAfgelopen4Weken,MatigVeelRegieOverEigenLeven,Eenzaam,ErnstigZeerErnstigEenzaam,EmotioneelEenzaam,SociaalEenzaam,Mantelzorger,Vrijwilligerswerk,MoeiteMetRondkomen
0,18 jaar of ouder,Waarde,2020,BU16800000,81.3,47.5,47.8,1.6,46.4,53.2,15.3,13.6,40.0,80.7,7.0,5.8,34.4,31.8,4.7,30.3,13.6,4.0,3.9,9.0,32.8,3.7,12.2,92.5,41.9,7.6,21.9,26.7,14.4,29.6,7.9
1,18 jaar of ouder,Waarde,2020,BU16800009,84.4,50.0,48.3,1.6,51.9,49.1,12.4,12.5,42.2,85.0,7.2,7.4,32.3,26.9,3.2,25.3,8.9,3.2,2.8,4.7,27.3,2.3,9.9,94.2,39.3,5.5,17.9,26.3,17.1,35.5,6.2
2,18 jaar of ouder,Waarde,2020,BU16800100,81.8,49.2,45.4,1.8,44.9,54.4,15.4,15.7,39.3,81.4,8.3,7.2,31.6,30.3,4.3,28.7,11.8,3.9,3.3,6.9,34.7,4.1,13.2,92.8,41.6,7.8,21.8,28.7,14.7,31.5,10.5
3,18 jaar of ouder,Waarde,2020,BU16800109,82.1,49.0,43.8,1.7,47.4,52.7,14.3,13.1,42.4,82.2,7.2,8.3,34.3,32.3,4.7,30.7,11.5,4.1,3.4,6.6,31.4,3.3,12.0,92.5,41.7,7.5,20.7,29.8,14.8,32.3,9.3
4,18 jaar of ouder,Waarde,2020,BU16800200,81.9,46.8,46.2,1.5,46.6,51.4,14.8,13.3,40.8,81.0,6.5,6.9,33.6,32.8,4.7,30.8,13.3,4.1,3.3,8.6,31.1,3.7,11.1,93.0,43.3,6.7,19.9,30.5,16.5,33.1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13803,18 jaar of ouder,Waarde,2020,BU01935140,82.5,54.9,52.2,1.9,48.3,51.0,14.9,16.4,40.8,81.9,8.2,6.1,31.9,28.7,4.1,27.6,11.8,3.5,4.3,6.6,45.5,4.7,17.0,92.5,42.8,8.6,24.9,26.1,15.3,34.6,10.6
13804,18 jaar of ouder,Waarde,2020,BU01935150,83.1,55.1,51.3,1.7,46.8,52.7,15.7,15.9,42.0,81.8,7.9,6.1,31.8,28.3,4.0,27.3,11.3,3.4,4.0,6.0,44.4,4.5,16.2,93.1,42.3,8.4,23.9,26.2,16.0,34.0,10.2
13805,18 jaar of ouder,Waarde,2020,BU01935160,92.3,63.4,71.2,1.7,59.0,40.7,9.8,9.1,33.5,91.0,9.2,6.4,23.0,18.1,1.6,17.9,7.2,2.6,2.2,3.0,34.7,2.6,16.1,97.2,28.5,4.1,14.5,16.3,14.8,47.1,3.9
13806,18 jaar of ouder,Waarde,2020,BU01935200,84.3,50.1,45.7,1.7,48.1,50.9,13.8,15.6,37.7,83.2,10.6,7.8,29.0,27.3,3.1,25.7,10.2,3.2,3.3,5.2,41.3,3.8,12.7,94.0,39.4,6.5,20.5,24.5,14.6,40.5,8.6


In [54]:
#============================== Merge Gezondheid 2020 met Kerncijfers 2020 =====================================
df_2020 = pd.merge(df_2020, gezondheid_2020, on="Codering", how="left").reset_index(drop=True)
df_2020.columns.name = "Dataset 2020"
#==================================================================================================================

In [55]:
df_2020

Dataset 2020,ID,WijkenEnBuurten,Gemeentenaam,SoortRegio,Codering,IndelingswijzigingWijkenEnBuurten,AantalInwoners,Mannen,Vrouwen,k0Tot15Jaar,k15Tot25Jaar,k25Tot45Jaar,k45Tot65Jaar,k65JaarOfOuder,Ongehuwd,Gehuwd,Gescheiden,Verweduwd,WestersTotaal,NietWestersTotaal,Marokko,NederlandseAntillenEnAruba,Suriname,Turkije,OverigNietWesters,GeboorteTotaal,GeboorteRelatief,SterfteTotaal,SterfteRelatief,HuishoudensTotaal,Eenpersoonshuishoudens,HuishoudensZonderKinderen,HuishoudensMetKinderen,GemiddeldeHuishoudensgrootte,Bevolkingsdichtheid,Woningvoorraad,GemiddeldeWOZWaardeVanWoningen,PercentageEengezinswoning,PercentageMeergezinswoning,PercentageBewoond,PercentageOnbewoond,Koopwoningen,HuurwoningenTotaal,InBezitWoningcorporatie,InBezitOverigeVerhuurders,EigendomOnbekend,BouwjaarVoor2000,BouwjaarVanaf2000,GemiddeldElektriciteitsverbruikTotaal,EnergieVerbruikAppartement,EnergieVerbruikTussenwoning,EnergieVerbruikHoekwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,EnergieVerbruikHuurwoning,EnergieVerbruikEigenWoning,GemiddeldAardgasverbruikTotaal,AardgasVerbruikAppartement,AardgasVerbruikTussenwoning,AardgasVerbruikHoekwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AardgasVerbruikHuurwoning,AardgasVerbruikEigenWoning,PercentageWoningenMetStadsverwarming,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OpleidingsniveauHoog,Nettoarbeidsparticipatie,PercentageWerknemers,PercentageZelfstandigen,AantalInkomensontvangers,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,k40PersonenMetLaagsteInkomen,k20PersonenMetHoogsteInkomen,GemGestandaardiseerdInkomenVanHuish,k40HuishoudensMetLaagsteInkomen,k20HuishoudensMetHoogsteInkomen,HuishoudensMetEenLaagInkomen,HuishOnderOfRondSociaalMinimum,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,MediaanVermogenVanParticuliereHuish,PersonenPerSoortUitkeringBijstand,PersonenPerSoortUitkeringAO,PersonenPerSoortUitkeringWW,PersonenPerSoortUitkeringAOW,JongerenMetJeugdzorgInNatura,PercentageJongerenMetJeugdzorg,WmoClienten,WmoClientenRelatief,BedrijfsvestigingenTotaal,ALandbouwBosbouwEnVisserij,BFNijverheidEnEnergie,GIHandelEnHoreca,HJVervoerInformatieEnCommunicatie,KLFinancieleDienstenOnroerendGoed,MNZakelijkeDienstverlening,OQOverheidOnderwijsEnZorg,RUCultuurRecreatieOverigeDiensten,PersonenautoSTotaal,PersonenautoSBrandstofBenzine,PersonenautoSOverigeBrandstof,PersonenautoSPerHuishouden,PersonenautoSNaarOppervlakte,Motorfietsen,AfstandTotHuisartsenpraktijk,AfstandTotGroteSupermarkt,AfstandTotKinderdagverblijf,AfstandTotSchool,ScholenBinnen3Km,OppervlakteTotaal,OppervlakteLand,OppervlakteWater,MeestVoorkomendePostcode,Dekkingspercentage,MateVanStedelijkheid,Omgevingsadressendichtheid,afw,fys,lbm,onv,soc,vrz,won,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,AfstandTotHuisartsenpost,AfstandTotApotheek,AfstandTotZiekenhuis,Ziekenhuis_Binnen_5km,Ziekenhuis_Binnen_10km,Ziekenhuis_Binnen_20km,AfstandTotZiekenhuis.1,Ziekenhuis_SEH_Binnen_5km,Ziekenhuis_SEH_Binnen_10km,Ziekenhuis_SEH_Binnen_20km,Supermarkt_Binnen_1km,Supermarkt_Binnen_3km,Supermarkt_Binnen_5km,AfstandTotOvDagelLevensmiddelen,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,AfstandTotWarenhuis,Warenhuis_Binnen_5km,Warenhuis_Binnen_10km,Warenhuis_Binnen_20km,AfstandTotCafeED,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,AfstandTotCafetariaED,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,AfstandTotRestaurant,Restaurant_Binnen_1km,Restaurant_Binnen_3km,Restaurant_Binnen_5km,AfstandTotHotelED,Hotel_Binnen_5km,Hotel_Binnen_10km,Hotel_Binnen_20km,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,AfstandTotBuitenschoolseOpvang,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,AfstandTotSchool.1,VoortgezetOnderwijs_Binnen_3km,VoortgezetOnderwijs_Binnen_5km,VoortgezetOnderwijs_Binnen_10km,AfstandTotSchool.2,VMBO_Binne

---
### Merge
---
- 2021-2022
- 2020

In [56]:
#======================================== Check Municipality Fusion =============================================== 
df_2020["Codering"] = df_2020["Codering"].apply(convert_codering, args=(2020, 2021))
matching_2020_2022 = df_2020[df_2020["Codering"].isin(df_2021_2022.index)]
#==================================================================================================================

#====================================== Merge 2020 with (2022 & 2021) =============================================
df_2020_2022 = df_2021_2022.combine_first(matching_2020_2022.set_index("Codering"))
df_2020_2022.columns.name = "Dataset 2020-2022"
contribution_dict[2020] = np.prod(df_2020_2022.shape) - df_2020_2022.isna().sum().sum() - contribution_dict[2022] - contribution_dict[2021]
#==================================================================================================================

---
# 2019

### Load DataFrames

- 2019 - CBS Kerncijfers  - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=84583NED

---

In [57]:
#====================================== Read & Clean the 2021 Datasets ============================================
kerncijfers_buurt_2019 = rename_c_kerncijfers(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2019/CBS Buurtkerncijfers.gzip")))
kerncijfers_buurt_2019.columns.name = "Kerncijfers 2019"
#==================================================================================================================

In [58]:
show_info([kerncijfers_buurt_2019])

Info Dataset Kerncijfers 2019
- Total Values:                         1604092
- Total Missings:                       216077
- % Missing:                            13.47%
- Dataset Dimensions:                   (13594, 118)



In [59]:
df_2019 = kerncijfers_buurt_2019.copy(deep=True)

In [60]:
#======================================== Check Municipality Fusion =============================================== 
df_2019["Codering"] = df_2019["Codering"].apply(convert_codering, args=(2019, 2020))
matching_2019_2022 = df_2019[df_2019["Codering"].isin(df_2020_2022.index)]
#==================================================================================================================

#======================================= Merge 2019 with (2020-2022) ==============================================
df_2019_2022 = df_2020_2022.combine_first(matching_2019_2022.set_index("Codering"))
df_2019_2022.columns.name = "Dataset 2019-2022"
contribution_dict[2019] = np.prod(df_2019_2022.shape) - df_2019_2022.isna().sum().sum() - contribution_dict[2022] - contribution_dict[2021] - contribution_dict[2020]
#==================================================================================================================

---
# Datasets 2018

### Load DataFrames

- 2018 - CBS Kerncijfers  - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=84286NED

---

In [61]:
#====================================== Read & Clean the 2018 Datasets ============================================
kerncijfers_buurt_2018 = rename_c_kerncijfers(clean_open_data(pd.read_parquet(path="../../Data/Location/CBS/2018/CBS Buurtkerncijfers.gzip")))
kerncijfers_buurt_2018.columns.name = "Kerncijfers 2018"
#==================================================================================================================

In [62]:
show_info([kerncijfers_buurt_2018])

Info Dataset Kerncijfers 2018
- Total Values:                         1569990
- Total Missings:                       211545
- % Missing:                            13.47%
- Dataset Dimensions:                   (13305, 118)



In [63]:
df_2018 = kerncijfers_buurt_2018.copy(deep=True)

In [64]:
#======================================= Merge 2018 with (2019-2022) ==============================================
matching_2018_2022 = df_2018[df_2018["Codering"].isin(df_2019_2022.index)]
df_2018_2022 = df_2019_2022.combine_first(matching_2018_2022.set_index("Codering"))
df_2018_2022.columns.name = "Dataset 2018-2022"
contribution_dict[2018] = np.prod(df_2018_2022.shape) - df_2018_2022.isna().sum().sum() - contribution_dict[2022] - contribution_dict[2021] - contribution_dict[2020] - contribution_dict[2019]
contribution_dict["missing"] = df_2018_2022.isna().sum().sum()
#==================================================================================================================

In [65]:
print(f"Contribution 2022 = {contribution_dict[2022]} {round(contribution_dict[2022]/ sum(contribution_dict.values()) * 100, 2)}")
print(f"Contribution 2021 = {contribution_dict[2021]} {round(contribution_dict[2021]/ sum(contribution_dict.values()) * 100, 2)}")
print(f"Contribution 2020 = {contribution_dict[2020]} {round(contribution_dict[2020]/ sum(contribution_dict.values()) * 100, 2)}")
print(f"Contribution 2019 = {contribution_dict[2019]} {round(contribution_dict[2019]/ sum(contribution_dict.values()) * 100, 2)}")
print(f"Contribution 2018 = {contribution_dict[2018]} {round(contribution_dict[2018]/ sum(contribution_dict.values()) * 100, 2)}")
print(f"Missing values = {df_2018_2022.isna().sum().sum()} {round(contribution_dict['missing']/ sum(contribution_dict.values()) * 100, 2)}")

Contribution 2022 = 964941 26.02
Contribution 2021 = 1714995 46.25
Contribution 2020 = 558251 15.05
Contribution 2019 = 11991 0.32
Contribution 2018 = 31869 0.86
Missing values = 426056 11.49


In [66]:
df_2018_2022

Dataset 2018-2022,ALandbouwBosbouwEnVisserij,AantalInkomensontvangers,AantalInstallatiesBijWoningen,AantalInwoners,AardgasVerbruikAppartement,AardgasVerbruikEigenWoning,AardgasVerbruikHoekwoning,AardgasVerbruikHuurwoning,AardgasVerbruikTussenwoning,AardgasVerbruikTweeOnderEenKapWoning,AardgasVerbruikVrijstaandeWoning,AfstandTotApotheek,AfstandTotAttractie,AfstandTotBelangrijkOverstapstation,AfstandTotBibliotheek,AfstandTotBioscoop,AfstandTotBrandweerkazerne,AfstandTotBuitenschoolseOpvang,AfstandTotCafeED,AfstandTotCafetariaED,AfstandTotGroteSupermarkt,AfstandTotHotelED,AfstandTotHuisartsenpost,AfstandTotHuisartsenpraktijk,AfstandTotKinderdagverblijf,AfstandTotKunstijsbaan,AfstandTotMuseum,AfstandTotOpritHoofdverkeersweg,AfstandTotOvDagelLevensmiddelen,AfstandTotPodiumkunstenTotaal,AfstandTotPoppodium,AfstandTotRestaurant,AfstandTotSauna,AfstandTotSchool,AfstandTotSchool.1,AfstandTotSchool.2,AfstandTotSchool.3,AfstandTotTreinstationsTotaal,AfstandTotWarenhuis,AfstandTotZiekenhuis,AfstandTotZiekenhuis.1,AfstandTotZonnebank,AfstandTotZwembad,Attractie_Binnen_10km,Attractie_Binnen_20km,Attractie_Binnen_50km,BFNijverheidEnEnergie,B_Opvang_Binnen_1km,B_Opvang_Binnen_3km,B_Opvang_Binnen_5km,Basisschool_Binnen_1km,Basisschool_Binnen_3km,Basisschool_Binnen_5km,BedrijfsvestigingenTotaal,BeperkingInBewegen,BeperkingInHoren,BeperkingInZien,BeperktVanwegeGezondheid,Berovingen,Bevolkingsdichtheid,Bioscoop_Binnen_10km,Bioscoop_Binnen_20km,Bioscoop_Binnen_5km,BouwjaarVanaf2000,BouwjaarVoor2000,Cafe_Binnen_1km,Cafe_Binnen_3km,Cafe_Binnen_5km,Cafetaria_Binnen_1km,Cafetaria_Binnen_3km,Cafetaria_Binnen_5km,Dekkingspercentage,Drinker,Drogist_Binnen_1km,Drogist_Binnen_3km,Drogist_Binnen_5km,EenOfMeerLangdurigeAandoeningen,EenOfMeerLichamelijkeBeperkingen,Eenpersoonshuishoudens,Eenzaam,EigendomOnbekend,EmotioneelEenzaam,EnergieVerbruikAppartement,EnergieVerbruikEigenWoning,EnergieVerbruikHoekwoning,EnergieVerbruikHuurwoning,EnergieVerbruikTussenwoning,EnergieVerbruikTweeOnderEenKapWoning,EnergieVerbruikVrijstaandeWoning,ErnstigBeperktVanwegeGezondheid,ErnstigOvergewicht,ErnstigZeerErnstigEenzaam,ErvarenGezondheidGoedZeerGoed,GIHandelEnHoreca,GeboorteRelatief,GeboorteTotaal,Gehuwd,GemGestandaardiseerdInkomenVanHuish,Gemeentenaam,GemiddeldAardgasverbruikTotaal,GemiddeldElektriciteitsverbruikTotaal,GemiddeldInkomenPerInkomensontvanger,GemiddeldInkomenPerInwoner,GemiddeldeHuishoudensgrootte,GemiddeldeWOZWaardeVanWoningen,GemiddeldeWoningwaarde,GeregistreerdeOverlast,Gescheiden,Geweld,GeweldsEnSeksueleMisdrijven,HAVO_VWO_Binnen_10km,HAVO_VWO_Binnen_3km,HAVO_VWO_Binnen_5km,HJVervoerInformatieEnCommunicatie,HeelVeelStressInAfgelopen4Weken,HoogRisicoOpAngstOfDepressie,Hotel_Binnen_10km,Hotel_Binnen_20km,Hotel_Binnen_5km,Huisarts_Binnen_1km,Huisarts_Binnen_3km,Huisarts_Binnen_5km,HuishOnderOfRondSociaalMinimum,HuishoudensMetEenLaagInkomen,HuishoudensMetKinderen,HuishoudensTot110VanSociaalMinimum,HuishoudensTot120VanSociaalMinimum,HuishoudensTotaal,HuishoudensZonderKinderen,HuurwoningenTotaal,ID,InBezitOverigeVerhuurders,InBezitWoningcorporatie,Inbraken,IndelingswijzigingWijkenEnBuurten,JongerenMetJeugdzorgInNatura,KLFinancieleDienstenOnroerendGoed,Kinderdagverblijf_Binnen_1km,Kinderdagverblijf_Binnen_3km,Kinderdagverblijf_Binnen_5km,Koopwoningen,KunstPodium_Binnen_10km,KunstPodium_Binnen_20km,KunstPodium_Binnen_5km,LangdurigeZiekteEnBeperkt,Leeftijd,MNZakelijkeDienstverlening,Mannen,Mantelzorger,Marges,Marokko,MateVanStedelijkheid,MatigHoogRisicoOpAngstOfDepressie,MatigVeelRegieOverEigenLeven,MediaanVermogenVanParticuliereHuish,MeestVoorkomendePostcode,MoeiteMetRondkomen,Motorfietsen,Museum_Binnen_10km,Museum_Binnen_20km,Museum_Binnen_5km,NederlandseAntillenEnAruba,Nettoarbeidsparticipatie,NietWestersTotaal,NormaalGewicht,OQOverheidOnderwijsEnZorg,Omgevingsadressendichtheid,Ondergewicht,Ongehuwd,OpgesteldVermogenVanZonnepanelen,OpleidingsniveauHoog,OpleidingsniveauLaag,OpleidingsniveauMiddelbaar,OppervlakteLand,OppervlakteTotaal,OppervlakteWater,Overgewicht,

In [67]:
df_2018_2022['Afstand tot school'] = df_2018_2022[['AfstandTotSchool.1', 'AfstandTotSchool.2', 'AfstandTotSchool.3', 'AfstandTotBuitenschoolseOpvang']].min(axis=1)
df_2018_2022.drop(columns=['AfstandTotSchool.1', 'AfstandTotSchool.2', 'AfstandTotSchool.3', 'AfstandTotBuitenschoolseOpvang'], inplace=True)

df_2018_2022['PersonenPerSoortUitkering'] = df_2018_2022[['PersonenPerSoortUitkeringAO', 'PersonenPerSoortUitkeringAOW', 'PersonenPerSoortUitkeringBijstand', 'PersonenPerSoortUitkeringWW']].sum(axis=1)
df_2018_2022.drop(columns=['PersonenPerSoortUitkeringAO', 'PersonenPerSoortUitkeringAOW', 'PersonenPerSoortUitkeringBijstand', 'PersonenPerSoortUitkeringWW'], inplace=True)

subset = ["AantalInwoners", "GemiddeldeWOZWaardeVanWoningen", "MateVanStedelijkheid", "AfstandTotBelangrijkOverstapstation", "AfstandTotOpritHoofdverkeersweg",
          "AfstandTotGroteSupermarkt", "BedrijfsvestigingenTotaal", "AantalInkomensontvangers", "Koopwoningen", "GemiddeldInkomenPerInwoner",
          "GemiddeldeHuishoudensgrootte", "k40PersonenMetLaagsteInkomen", "k20PersonenMetHoogsteInkomen", "k0Tot15Jaar", "k15Tot25Jaar",
            "k25Tot45Jaar", "k45Tot65Jaar", "k65JaarOfOuder", "Supermarkt_Binnen_1km", "PercentageEengezinswoning",
            "Eenpersoonshuishoudens", "HuishoudensMetKinderen", 'PersonenPerSoortUitkering', 'Afstand tot school']

df_2018_2022 = df_2018_2022[subset]
df_2018_2022

/var/folders/nx/t23l2x7x65l5f929y7y2ghlw0000gn/T/ipykernel_272/1152209594.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2018_2022['Afstand tot school'] = df_2018_2022[['AfstandTotSchool.1', 'AfstandTotSchool.2', 'AfstandTotSchool.3', 'AfstandTotBuitenschoolseOpvang']].min(axis=1)
/var/folders/nx/t23l2x7x65l5f929y7y2ghlw0000gn/T/ipykernel_272/1152209594.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2018_2022['PersonenPerSoortUitkering'] = df_2018_2022[['PersonenPerSoortUitkeringAO', 'PersonenPerSoortUitkeringAOW

Dataset 2018-2022,AantalInwoners,GemiddeldeWOZWaardeVanWoningen,MateVanStedelijkheid,AfstandTotBelangrijkOverstapstation,AfstandTotOpritHoofdverkeersweg,AfstandTotGroteSupermarkt,BedrijfsvestigingenTotaal,AantalInkomensontvangers,Koopwoningen,GemiddeldInkomenPerInwoner,GemiddeldeHuishoudensgrootte,k40PersonenMetLaagsteInkomen,k20PersonenMetHoogsteInkomen,k0Tot15Jaar,k15Tot25Jaar,k25Tot45Jaar,k45Tot65Jaar,k65JaarOfOuder,Supermarkt_Binnen_1km,PercentageEengezinswoning,Eenpersoonshuishoudens,HuishoudensMetKinderen,PersonenPerSoortUitkering,Afstand tot school
Codering,,,,,,,,,,,,,,,,,,,,,,,,
BU00140000,4480,247.0,1.0,1.3,1.7,0.4,1030.0,3800.0,12.0,22.1,1.2,59.8,10.5,75,2190,1465,465,290,9.7,13.0,2985,95,530.0,0.4
BU00140001,6630,251.0,1.0,0.8,1.3,0.3,1730.0,5500.0,12.0,22.7,1.2,61.4,9.8,115,3160,2205,670,485,7.6,9.0,4430,140,850.0,0.4
BU00140002,4150,216.0,1.0,1.6,1.4,0.5,620.0,3500.0,11.0,21.5,1.3,62.0,9.8,120,1765,1435,455,375,6.5,13.0,2615,135,670.0,0.4
BU00140003,1710,247.0,1.0,0.9,1.3,0.2,320.0,1500.0,13.0,24.8,1.3,52.6,13.6,45,685,660,150,175,7.2,11.0,1095,45,260.0,0.7
BU00140004,10,NaN,1.0,1.4,1.4,0.4,5.0,0.0,NaN,NaN,1.8,NaN,NaN,0,5,0,5,5,7.5,NaN,5,5,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BU19910603,480,460.0,5.0,26.2,3.2,2.7,70.0,400.0,90.0,NaN,2.7,42.7,20.5,70,70,80,150,110,0.0,84.0,30,80,120.0,2.7
BU19910604,375,500.0,5.0,25.0,2.0,2.8,75.0,300.0,84.0,NaN,2.7,41.4,19.9,60,55,60,120,80,0.0,95.0,30,65,90.0,2.7
BU19910605,365,560.0,5.0,25.3,2.3,1.9,40.0,200.0,79.0,NaN,1.9,43.9,23.8,35,40,55,155,85,0.0,95.0,105,40,70.0,2.0


In [68]:
show_info(df_list=[df_2018_2022])

Info Dataset Dataset 2018-2022
- Total Values:                         343608
- Total Missings:                       20549
- % Missing:                            5.98%
- Dataset Dimensions:                   (14317, 24)



In [69]:
df_2018_2022.rename(columns={
                     'PersonenPerSoortUitkering': 'n_res_benefits',
                     'AantalInwoners': "population",
                     "Afstand tot school": 'd_school',
                     'AfstandTotBelangrijkOverstapstation': 'd_transfer',
                     'AfstandTotOpritHoofdverkeersweg': 'd_mainstreet',
                     'BedrijfsvestigingenTotaal': 'n_business',
                     'AantalInkomensontvangers': 'n_earners',
                     'AfstandTotGroteSupermarkt': 'd_supermarket',
                     'Bevolkingsdichtheid': 'population_density',
                     'GemiddeldInkomenPerInwoner': 'avg_income',
                     'PercentageEengezinswoning': '%_single_family_homes',
                     'k20PersonenMetHoogsteInkomen': 'k20_high_income',
                     'HuishoudensMetKinderen': 'households_with_children',
                     'Eenpersoonshuishoudens': 'single_households',
                     'GemiddeldeHuishoudensgrootte': 'household_size',
                     'k40PersonenMetLaagsteInkomen': 'k40_low_income',
                     'GemiddeldeWOZWaardeVanWoningen': 'WOZ',
                     'Koopwoningen': 'n_owner_occupied',
                     'MateVanStedelijkheid': 'urbanity_score',
                     'Supermarkt_Binnen_1km': 'supermarket_1km',
                     'k0Tot15Jaar': 'k0_k15',
                     'k15Tot25Jaar': 'k15_k25',
                     'k25Tot45Jaar': 'k25_k45',
                     'k45Tot65Jaar': 'k45_k65',
                     'k65JaarOfOuder':  'k65+'}, inplace=True)


#'Huurprijs per m2':'rent_price',
                     # 'VVO': 'GLA',

/var/folders/nx/t23l2x7x65l5f929y7y2ghlw0000gn/T/ipykernel_272/2271685728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018_2022.rename(columns={


In [70]:
df_2018_2022

Dataset 2018-2022,population,WOZ,urbanity_score,d_transfer,d_mainstreet,d_supermarket,n_business,n_earners,n_owner_occupied,avg_income,household_size,k40_low_income,k20_high_income,k0_k15,k15_k25,k25_k45,k45_k65,k65+,supermarket_1km,%_single_family_homes,single_households,households_with_children,n_res_benefits,d_school
Codering,,,,,,,,,,,,,,,,,,,,,,,,
BU00140000,4480,247.0,1.0,1.3,1.7,0.4,1030.0,3800.0,12.0,22.1,1.2,59.8,10.5,75,2190,1465,465,290,9.7,13.0,2985,95,530.0,0.4
BU00140001,6630,251.0,1.0,0.8,1.3,0.3,1730.0,5500.0,12.0,22.7,1.2,61.4,9.8,115,3160,2205,670,485,7.6,9.0,4430,140,850.0,0.4
BU00140002,4150,216.0,1.0,1.6,1.4,0.5,620.0,3500.0,11.0,21.5,1.3,62.0,9.8,120,1765,1435,455,375,6.5,13.0,2615,135,670.0,0.4
BU00140003,1710,247.0,1.0,0.9,1.3,0.2,320.0,1500.0,13.0,24.8,1.3,52.6,13.6,45,685,660,150,175,7.2,11.0,1095,45,260.0,0.7
BU00140004,10,NaN,1.0,1.4,1.4,0.4,5.0,0.0,NaN,NaN,1.8,NaN,NaN,0,5,0,5,5,7.5,NaN,5,5,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BU19910603,480,460.0,5.0,26.2,3.2,2.7,70.0,400.0,90.0,NaN,2.7,42.7,20.5,70,70,80,150,110,0.0,84.0,30,80,120.0,2.7
BU19910604,375,500.0,5.0,25.0,2.0,2.8,75.0,300.0,84.0,NaN,2.7,41.4,19.9,60,55,60,120,80,0.0,95.0,30,65,90.0,2.7
BU19910605,365,560.0,5.0,25.3,2.3,1.9,40.0,200.0,79.0,NaN,1.9,43.9,23.8,35,40,55,155,85,0.0,95.0,105,40,70.0,2.0


In [71]:
df_2018_2022.to_parquet(path="../../Data/Location/Merged/Dataset.gzip", compression="gzip")